In [1]:
import os
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import box
from shapely.geometry import Point
import rasterio
import geopandas as gpd
import json
import requests
import urllib
import random
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
# import gdal
# from osgeo import gdal

import numpy as np
import math
import shapely.geometry.point

In [3]:
# Weighted Average Interpolation module
# x and y are the unknown coordinates
# xCoords, yCoords, elevs are the known coordinates of the nehbor pixels
# order is the power of distance in the inverse distance that is used as a weight in IDW

def IDW(x, y, xCoords, yCoords, elevs, order=2):

    d =  np.zeros(len(xCoords))
    w =  np.zeros(len(xCoords))

    # if the distance from the unknown point to the central pixel is less than 1 cm, the evlevation of central pixel is assigned without using IDW
    if  math.sqrt((x-xCoords[0])**2 +(y-yCoords[0])**2) < 0.01:
        z = elevs[0]
    else:
        for i in range(len(xCoords)):
            d[i] = math.sqrt((x-xCoords[i])**2 +(y-yCoords[i])**2)
            w[i] = 1.0 / (d[i]**order)
        z = (np.sum(elevs*w)) / float(np.sum(w))

    return z

In [4]:
# This function get a 5*5 raster and return the m closest pixel to the point x,y located in the central pixel
def neibr(rasterBlock_x, rasterBlock_y, rasterBlock_elev, x, y, m):
    '''
    This function create the m closest pixel to the point(x,y)
    rasterBlock is the 5 by 5 matrix
    x,y is the location of sample points that needs to be interpolated
    '''
    if m ==4:
        if x <= rasterBlock_x[2,2]:
            if y >= rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,1], rasterBlock_x[1,1], rasterBlock_x[1,2]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,1], rasterBlock_y[1,1], rasterBlock_y[1,2]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,1], rasterBlock_elev[1,1], rasterBlock_elev[1,2]])
            elif y < rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,1], rasterBlock_x[3,1], rasterBlock_x[3,2]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,1], rasterBlock_y[3,1], rasterBlock_y[3,2]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,1], rasterBlock_elev[3,1], rasterBlock_elev[3,2]])
        elif x > rasterBlock_x[2,2]:
            if y >= rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [1,3], rasterBlock_x[1,2], rasterBlock_x[2,3]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [1,3], rasterBlock_y[1,2], rasterBlock_y[2,3]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [1,3], rasterBlock_elev[1,2], rasterBlock_elev[2,3]])
            elif y < rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,3], rasterBlock_x[3,2], rasterBlock_x[3,3]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,3], rasterBlock_y[3,2], rasterBlock_y[3,3]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,3], rasterBlock_elev[3,2], rasterBlock_elev[3,3]])
    return xCoor, yCoor, elev


In [5]:
def extractValue(x, y, dataset):
    '''
    This function extract value of a point from a raster
    :param x: x coordinate of point
    :param y: y coordinate of point
    :param dataset: a raster object imported by Rasterio
    :return: return the elevation of point
    '''
    val = dataset.sample([(x, y)])
    return list(val)[0][0]

def extractWindow(x, y, dataset, cellSize):
    '''
    This function aims to extract a 5*5 matrix from the input raster
    Also, this function convert the UTM coordinates to the local coordinate system of the extracted matrix
    :param x: x coordinate of point
    :param y: y coordinate of point
    :param dataset: a raster object imported by Rasterio
    :param cellSize: raster cell size
    :return: three 5*5 matrix (x, y, and elevation). Also the local coordinates of the input point
    '''
    row, col = dataset.index(x, y) # where the point is located in the raster
    rasterBlock_elev = dataset.read(1, window=((row-2, row+3), (col-2, col+3))) # extract the 5*5 raster block
    
    #find the upper left corner coordinates of the extracted 5*5 matrix
    # ulp = dataset.affine * (col-2, row-2)#src.affine * (col, row)
    ulp = dataset.transform * (col-2, row-2)
    ulpCX= ulp[0] + (cellSize/2.0) # coordinates of central point of the upper left corner pixel
    ulpCY= ulp[1] - (cellSize/2.0)

    # coordinates of the  5*5 matrix
    X = np.arange(ulpCX, ulpCX+cellSize*5, cellSize)
    Y = np.arange(ulpCY, ulpCY-cellSize*5, -cellSize)
    X, Y = np.meshgrid(X, Y)
    
    # A local coordinate system is created; the center of the 5*5 matrix is (0,0)
    x=x-X[2,2]
    y=y-Y[2,2]

    rasterBlock_x = X - X[2,2]
    rasterBlock_y = Y - Y[2,2]
    
    return x, y, rasterBlock_x, rasterBlock_y, rasterBlock_elev


In [6]:
from shapely.geometry import Point
import os
import geopandas as gpd
import pandas as pd
import rasterio
# Set the current workspace (all of the input DEMs are in this folder)

# Input data (DEMs): they are in ESRI Grid format
'''THIS ALWAYS NEEDS TO BE CHANGED IF YOU CHANGE A DEM'''
DEMs = ['dem_utm_45001']
resolutions = [10]


In [7]:
def eleva(dataframe):
    '''DO NOT FORGET TO CHANGE DEM HERE IF YOUR RE USING A DIFFERENT ONE
    I create a function out of this section since it gonna be used to extract 
    elevation from sampled points along our linestrings.
    
    dataframe = is the dataframe of points created , extracted by linestrings
    returns = a geodataframe with two new columns, elevation and interpolated elevation
    
    '''
    with rasterio.open('dem_utm_45001') as dem3m:
        samples = dataframe # randomPnts shapefile is imported as a geodataframe
        samples['elev3m'] = None # adding a new attribute "elev3m" to the geodataframe
        for index, row in samples.iterrows(): # Extract the elevation of each point from benchmark
            X = row['geometry'].x
            Y = row['geometry'].y
            samples.at[index, 'elev10m'] = extractValue(X,Y,dem3m).astype('float64')

    # Create the residual geodataframe. This dataframe is completed at the end
    residuals = samples.copy()
    '''Weighted average on extracted points of linestring'''
    # Methods used for calculating surface area
    #     methods = ['WP', 'WA4', 'Li3', 'BiLi4', 'BiQ9', 'BiC16']
    methods = ['WA4']

    #Add new feilds to the the "samples" geodataframe based on all combinations of resolutions and methods
    fields = []
    for res in resolutions:
        for mth in methods:
            samples[mth + str(int(res))] = None
            fields = fields + [mth + str(res)]

    for dem in DEMs:  # for each resolution surface-adjusted elevations are estimated for various interpolation methods
        with rasterio.open(dem) as src:
            cellSize = src.transform[0] # get actual DEM resolution fo computations
            res = resolutions[DEMs.index(dem)] # get nominal DEM resolution for labeling
            for index, row in samples.iterrows(): # iterating through all of the points
                X = row['geometry'].x
                Y = row['geometry'].y
                # The extractWindow function in the findValue module returns the 5*5 elevation matrix for the DEM based on the point
                # The coordinate of the central pixel is (0,0). x,y are the coordinates of the point in this local coordinate system
                x, y, rasterBlock_x, rasterBlock_y, rasterBlock_elev = extractWindow(X, Y, src, cellSize)

                try:
                    xCoor4, yCoor4, elev4 = neibr (rasterBlock_x, rasterBlock_y, rasterBlock_elev, x, y, 4)
                    samples.at[index, 'WA4'+ str(res)] = IDW(x, y, xCoor4, yCoor4, elev4, 2).astype('float64')

                except IndexError:
                    print ('index out of bounds'+str(index))
                    continue
    samples=gpd.GeoDataFrame(samples,geometry=samples['geometry'])
    return(samples)

In [8]:
%pwd

'C:\\Users\\zenus\\Downloads\\Resilio_box_desktop\\Proposal\\figshare_Dijks'

In [9]:
'''Boulder roads and random points'''
import geopandas as gpd
import shapely
from shapely.geometry import LineString
from shapely.geometry import *
import networkx as nx   
import pandas as pd

ex = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/cleanup_folder/rds_split_true.shp'

ex1 = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/Adjusted_rds.shp'
extnd= gpd.read_file(ex)
simplify_extend = gpd.GeoDataFrame(extnd,geometry=extnd['geometry'])

rndm = gpd.read_file('rndm10m_pnts.shp')

farms_ = 'Farm_point.shp'
frms_ = gpd.read_file(farms_)
farm = gpd.GeoDataFrame(frms_, geometry=frms_['geometry'])

scor = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/decenter_boots/decenter_boots.shp'
points_output = gpd.read_file(scor)

rnd_a = 'Power_plant.shp'
valmont = gpd.read_file(rnd_a)

pixpi = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/pixpix_trueonly/pixpix_trueonly.shp'
pixpix = gpd.read_file(pixpi)

In [516]:
# ex1 = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/cleanup_folder/rds_split_true.shp'

# ex2 = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/pixpix_trueonly/pixpix_trueonly.shp'
ex = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/Seeing_ifthis_ispixpix/pixpix_trueonly/pixpix_trueonly.shp'

extnd= gpd.read_file(ex)
simplify_extend = gpd.GeoDataFrame(extnd,geometry=extnd['geometry'])

In [797]:
'''These two cell this and the below were used for surface adjusted distances for the linestring connecting all centroids
to closest road point'''
rndm10frmslink = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/rndm10_freq_Intersect.shp'
rndm10frms = gpd.read_file(rndm10frmslink)

In [798]:
'''For the surface adjusted distance of points from within zones to closest road point'''
linesfromfrm = 'C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
linestring_fromfrequentcentroid_10ms/linestring_fromfrequentcentroid_10ms.shp'

linesfrm = gpd.read_file(linesfromfrm)

In [789]:
# rndm10frms['neighb'] = rndm10frms['CID']

In [832]:
'''This cell is to populate the polylines with starting and ending point and the inbetween random 10 meter points'''
analysisoflines = []
for i,j in linesfrm.iterrows():
    empty = []
    empty.append(shapely.geometry.Point(j.geometry.coords[:][0]))
    '''When APN numbers match we dont need the intersect'''
#     setofrndms = rndm10frms.loc[np.where(rndm10frms['APN']==j.APN)]
    '''This intersection is needed when the APN numbers do not match'''
    setofrndms=rndm10frms.loc[np.where(j['geometry'].buffer(1).intersects(rndm10frms.geometry)==True)]
    empty.extend(setofrndms.geometry)
    
#     print(j.geometry.coords[:][1])
    empty.append(shapely.geometry.Point(j.geometry.coords[:][1]))
    completeset = gpd.GeoDataFrame(geometry =[shapely.geometry.LineString(empty)],crs = simplify_extend.crs)
    completeset['APN'] = j.APN
    completeset['neighb'] = j.neighb
    completeset['n_pnts'] = j.n_pnts
    completeset['error'] = j.error
    analysisoflines.append(completeset)
#     analysisoflines.append(gpd.GeoDataFrame(geometry = []))

In [833]:
analysi = pd.concat([k for k in analysisoflines], ignore_index= True)

analysiss=gpd.GeoDataFrame(analysi,geometry=analysi['geometry'],crs=simplify_extend.crs)   

In [835]:
# rst_ = rasterio.open('/content/drive/MyDrive/Proposal/figshare_Dijks/dembuff')#you had top export another format for rasterio to work

# r = gdal.Open("/content/drive/MyDrive/Proposal/figshare_Dijks/dembuff")
# band = r.GetRasterBand(1) #bands start at one
# a = band.ReadAsArray().astype(float)


def locate (xindex,yindex, raster):
  (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = raster.GetGeoTransform()
  x_coords = xindex * x_size + upper_left_x + (x_size / 2) #add half the cell size
  y_coords = yindex * y_size + upper_left_y + (y_size / 2) #to centre the point
  return(x_coords,y_coords)

In [837]:
# '''This is a copy of the cell below used to interpolate elevations for the linestrings that connect points from within
# candidate zones to closest points on road and surface adjust distance measurement'''
# concat = []
# crs = simplify_extend.crs
# yo=0
# concat1 = []
# name = []
# for i,g in analysiss.iterrows():
#     '''Stand_line has all data we need to pass through the surface adjusted part. Here we make use of function eleva, which for each 
#     point of the road net, it extracts the interpolated elevation value based on IDW in a new column called WA410.

#     tosave: the coordinates of each line connecting all 10m random points of a segment
#     note: the dataframe that is passed to function eleva
#     samples: the output of the function with the IDW stored in it
#     d: is the surface adjusted distance that is calculated for the lines
#     std: the standard deviation
#     mean: mean of elevation
#     saveL: the geodataframe that contains the new metrics
#     concat: all segments of the road network
#     its_adjusted: the final geodataframe that combines everything
#     '''
    
#     erasetosave=[]
#     arraysave=[]
#     if type(g['geometry'])!=shapely.geometry.MultiLineString:
#         tosave=[]
#         tosave.append(g['geometry'].coords[:])
#         arraysave.append(tosave)
#         yo+=1
   
#         erasetosave.append(shapely.geometry.LineString(tosave[0]))
        
#         for z in range(len(tosave)):
#             passing=[]
#             note=pd.DataFrame()
#             note=pd.DataFrame(tosave[z],columns=['x','y'])
#             # passing=[Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))]
# #             print passing
# #             print note
#             note['geometry']=gpd.GeoSeries([Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))])
# #             print note
#             '''The extracted points that create the linestring are used to extract the elevation'''
#             samples = eleva(note)
# #             print (samples)
#         #     samples.to_file(driver = 'ESRI Shapefile', filename = output+ r'\samples'+str(i)+'.shp') #Export estimated eleavtions as a shapefile
#             '''Calculating surface adjusted distance for the linestring'''
#             try:
#                 d=0
#                 for ii in range(len(samples['geometry'])-1):
#                     d += math.sqrt((samples['geometry'][ii].x-samples['geometry'][ii+1].x)**2 +(samples['geometry'][ii].y\
#                                                     -samples['geometry'][ii+1].y)**2 + (samples['WA410'][ii]-samples['WA410'][ii+1])**2)
#     #                 print d

#                 '''Euclidian distance between starting and ending point of linestring for comparisson'''
#                 # eud=0
#                 # eud += math.sqrt((tosave[z][0][0]-tosave[z][-1][0])**2 +(tosave[z][0][1]\
#                 #                                     -tosave[z][-1][1])**2)
#     #                 print eud
#                 mean=0
#                 for yy in samples['WA410']:
#                     mean+= yy
#                 mean = mean/samples.count()[0]
#                 # print (str('the mean is') + str(mean))
                
#                 dif=0
#                 for yy in samples['WA410']:
#                     dif+= (yy-mean)**2
#                 std = dif/samples.count()[0]
# #                 print ('standard is ' + str(std))
                
#             except: #in case something is wrong , you ll be able to see it in the outfile
#                 print ('there was an elevation error')
#                 eud=-9999
#                 d=-9999
#                 std = -9999
#             '''ta parakatw ta kanw comment in mono gia na kanw extract ta split roads pou den exoun name'''
#             saveL=gpd.GeoDataFrame(geometry = [erasetosave[z]],crs = crs)
#             # saveL['eud']=eud
#             saveL['planar']=erasetosave[z].length
#             saveL['surf_adj']=d
#             saveL['std']=std
#             saveL['mean']=mean
#             saveL['geometry']=erasetosave[z]
#             saveL['APN'] = g.APN
#             saveL['n_pnts'] = g.n_pnts
#             saveL['neighb'] = g.neighb
#             if name == []:
#                 # saveL['name']=pd.core.series.Series(g['name']) #Becareful here the name = fixed 
#                 # saveL['eud']=eud
#                 saveL['planar']=erasetosave[z].length
#                 saveL['surf_adj']=d
#                 saveL['std']=std
#                 saveL['mean']=mean
#                 saveL['geometry']=erasetosave[z]

#             crs=simplify_extend.crs
#             saveL=gpd.GeoDataFrame(saveL, geometry=saveL['geometry'],crs=crs)
# #             saveL.to_file(driver= 'ESRI Shapefile',filename = output+r'\saveL' +str(g['NAME'])+'.shp')
#             concat.append(saveL)

# problemad=pd.concat([k for k in concat], ignore_index= True)

# problemadj=gpd.GeoDataFrame(problemad,geometry=problemad['geometry'],crs=crs)   

# # ontop.to_file('simplify_net_adjusted')

In [838]:
# problemadj.to_file('adjusted_frequentlines_insidezones')

In [765]:
'''This cell deals with bringing the surface adjusted distance and combine it with the dataframe stand_lin
This cell has a long runtime, because we run through all the segments and all the random 10m sample points combined.'''
concat = []
crs = simplify_extend.crs
yo=0
concat1 = []
name = []
for i,g in simplify_extend.iterrows():
    '''Stand_line has all data we need to pass through the surface adjusted part. Here we make use of function eleva, which for each 
    point of the road net, it extracts the interpolated elevation value based on IDW in a new column called WA410.

    tosave: the coordinates of each line connecting all 10m random points of a segment
    note: the dataframe that is passed to function eleva
    samples: the output of the function with the IDW stored in it
    d: is the surface adjusted distance that is calculated for the lines
    std: the standard deviation
    mean: mean of elevation
    saveL: the geodataframe that contains the new metrics
    concat: all segments of the road network
    its_adjusted: the final geodataframe that combines everything
    '''
    
    erasetosave=[]
    arraysave=[]
    if type(g['geometry'])!=shapely.geometry.MultiLineString:
        tosave=[]
        tosave.append(g['geometry'].coords[:])
        arraysave.append(tosave)
        yo+=1
   
        erasetosave.append(shapely.geometry.LineString(tosave[0]))
        
        for z in range(len(tosave)):
            passing=[]
            note=pd.DataFrame()
            note=pd.DataFrame(tosave[z],columns=['x','y'])
            # passing=[Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))]
#             print passing
#             print note
            note['geometry']=gpd.GeoSeries([Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))])
#             print note
            '''The extracted points that create the linestring are used to extract the elevation'''
            samples = eleva(note)
            print (samples)
        #     samples.to_file(driver = 'ESRI Shapefile', filename = output+ r'\samples'+str(i)+'.shp') #Export estimated eleavtions as a shapefile
            '''Calculating surface adjusted distance for the linestring'''
            try:
                d=0
                for ii in range(len(samples['geometry'])-1):
                    d += math.sqrt((samples['geometry'][ii].x-samples['geometry'][ii+1].x)**2 +(samples['geometry'][ii].y\
                                                    -samples['geometry'][ii+1].y)**2 + (samples['WA410'][ii]-samples['WA410'][ii+1])**2)
    #                 print d

                '''Euclidian distance between starting and ending point of linestring for comparisson'''
                # eud=0
                # eud += math.sqrt((tosave[z][0][0]-tosave[z][-1][0])**2 +(tosave[z][0][1]\
                #                                     -tosave[z][-1][1])**2)
    #                 print eud
                mean=0
                for yy in samples['WA410']:
                    mean+= yy
                mean = mean/samples.count()[0]
                # print (str('the mean is') + str(mean))
                
                dif=0
                for yy in samples['WA410']:
                    dif+= (yy-mean)**2
                std = dif/samples.count()[0]
                print ('standard is ' + str(std))
                
            except: #in case something is wrong , you ll be able to see it in the outfile
                print ('there was an elevation error')
                eud=-9999
                d=-9999
                std = -9999
            '''ta parakatw ta kanw comment in mono gia na kanw extract ta split roads pou den exoun name'''
            saveL=gpd.GeoDataFrame(geometry = [erasetosave[z]],crs = crs)
            # saveL['eud']=eud
            saveL['planar']=erasetosave[z].length
            saveL['surf_adj']=d
            saveL['std']=std
            saveL['mean']=mean
            saveL['geometry']=erasetosave[z]
            saveL['CID'] = g.CID
            if name == []:
                # saveL['name']=pd.core.series.Series(g['name']) #Becareful here the name = fixed 
                # saveL['eud']=eud
                saveL['planar']=erasetosave[z].length
                saveL['surf_adj']=d
                saveL['std']=std
                saveL['mean']=mean
                saveL['geometry']=erasetosave[z]

            crs=simplify_extend.crs
            saveL=gpd.GeoDataFrame(saveL, geometry=saveL['geometry'],crs=crs)
#             saveL.to_file(driver= 'ESRI Shapefile',filename = output+r'\saveL' +str(g['NAME'])+'.shp')
            concat.append(saveL)

its_adjusted=pd.concat([k for k in concat], ignore_index= True)

its_adjusted=gpd.GeoDataFrame(its_adjusted,geometry=its_adjusted['geometry'],crs=crs)   

# ontop.to_file('simplify_net_adjusted')

In [515]:
its_adjusted.to_file('potentialadjustednet')

In [130]:
qdt = its_adjusted
# qdt['pix_pix'] = stand_lin['pix_pix']
qdt

,OBJECTID,eud,p2p,surf_adj,std,mean,Shape_Leng,geometry,planar,CID
0,1,30.863481,30.863481,30.866965,0.053762,1504.541655,30.863481,"LINESTRING (492102.724 4444784.457, 492099.000...",30.863481,1
1,2,3102.914504,3104.074833,3104.201673,19.327717,1526.326487,3104.074833,"LINESTRING (491961.202 4447478.886, 491946.824...",3104.074833,2
2,3,802.855938,802.857158,803.837232,165.310223,1678.533281,802.857158,"LINESTRING (476424.619 4427802.878, 476424.515...",802.857158,3
3,4,690.227385,694.190291,694.513899,8.249812,1635.844592,694.190291,"LINESTRING (476188.413 4429924.898, 476160.621...",694.190291,4
4,5,10.753368,10.753368,10.754445,0.005788,1635.028726,10.753368,"LINESTRING (476059.189 4430602.921, 476064.700...",10.753368,5
...,...,...,...,...,...,...,...,...,...,...
1390,1391,448.830575,451.432102,451.543529,5.427013,1656.598970,451.432102,"LINESTRING (488299.050 4419879.654, 488299.189...",451.432102,1391
1391,1392,398.803732,405.482861,405.884109,22.819649,1646.760539,405.482861,"LINESTRING (488270.321 4420327.564, 488271.088...",405.482861,1392
1392,1393,402.573106,408.746399,409.215415,33.680949,1648.115599,408.746399,"LINESTRING (488310.696 4420727.028, 488293.075...",408.746399,1393
1393,1394,18.348306,18.348306,18.348437,0.001205,1655.182085,18.348306,"LINESTRING (488285.770 4419879.264, 488285.486...",18.348306,1394


In [520]:
import networkx as nx 

In [521]:
detailgraph = nx.Graph()
for i,elrow in its_adjusted.iterrows():
#     print(elrow.geometry)
    for y in range(len(elrow.geometry.coords[:])-1):
#         print(y)
#         print(shapely.geometry.LineString([elrow.geometry.coords[:][y],elrow.geometry.coords[:][y+1]]).length)
        detailgraph.add_edge(elrow.geometry.coords[:][y], elrow.geometry.coords[:][y+1], weight = \
                       shapely.geometry.LineString([elrow.geometry.coords[:][y],elrow.geometry.coords[:][y+1]]).length )

In [522]:
'''Create three seperate graphs that contain the three different distance metrics'''
'''The term weight bellow specifies which distance each graph is going to be storing'''
newGraph = nx.Graph()
newGraph_adj = nx.Graph()
newGraph_pix = nx.Graph()
for i, elrow in its_adjusted.iterrows():
    newGraph.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1],\
                      weight=elrow['planar'] ,attr_dict=elrow[2:].to_dict())
    

for i, elrow in its_adjusted.iterrows():
    newGraph_adj.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1],\
                          weight=elrow['surf_adj'] ,attr_dict=elrow[2:].to_dict())
    
    
# for i, elrow in its_adjusted.iterrows():
#     newGraph_pix.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1], weight = elrow['pix_pix'],\
#                          attr_dict=elrow[1:].to_dict())

In [523]:
'''From the graphs you need to capture the nodes that are going to be used for Djikstra routine'''
'''Three seperate variables are created in order to make sure that all nodes are captured from each graph, although one set could also be used'''
checkNodes = [shapely.geometry.Point(i) for i in list(newGraph.nodes)]
checknodes = gpd.GeoDataFrame(geometry=checkNodes , crs=crs)

detailnodes = [shapely.geometry.Point(i) for i in list(detailgraph.nodes)]
detailednodes = gpd.GeoDataFrame(geometry=detailnodes , crs=crs)

checkNodes_adj = [shapely.geometry.Point(i) for i in list(newGraph_adj.nodes)]
checknodes_adj=gpd.GeoDataFrame(geometry=checkNodes_adj , crs=crs)

# checkNodes_pix = [shapely.geometry.Point(i) for i in list(newGraph_pix.nodes)]
# checknodes_pix = gpd.GeoDataFrame(geometry=checkNodes_pix , crs=its_adjusted.crs)

In [524]:
end = (valmont['geometry'][0].x,valmont['geometry'][0].y)

In [525]:
'''Since Valmont point is not part of the nodes created from the graphs, the point is replaced with the closest node from the graphs above'''
'''Create a buffer with radius 10m in this case and find out which node is the closest to the end point, Valmont power plant'''
buffclosest = valmont['geometry'][0].buffer(700)
for i in range(len(checkNodes)):
    if buffclosest.contains(checkNodes[i])==True:
        print(i,checkNodes[i])
        print(shapely.geometry.LineString([valmont['geometry'][0],checkNodes[i]]).length, 'meters away')

46395 POINT (482366.5647 4429400.874299997)
171.0168916839568 meters away
46396 POINT (482366.5951240035 4429400.874009317)
170.9864692618928 meters away
46397 POINT (482376.5988424402 4429400.778430253)
160.98333168543843 meters away
46398 POINT (482386.59949435014 4429400.682880489)
150.98339787018116 meters away
46399 POINT (482396.6080671642 4429400.587255048)
140.97571055686095 meters away
46400 POINT (482406.6144281309 4429400.491650738)
130.97043997776044 meters away
46401 POINT (482416.6292140363 4429400.395965932)
120.95700166555015 meters away
46402 POINT (482426.64501899667 4429400.281350134)
110.94309864383578 meters away
46403 POINT (482436.6566451574 4429400.146963244)
100.93417009966008 meters away
46404 POINT (482446.7618232975 4429400.011320604)
90.83251935453224 meters away
46405 POINT (482456.7864789013 4429399.876758819)
80.81245950216699 meters away
46406 POINT (482466.81349756196 4429399.742165318)
70.79159531300613 meters away
46407 POINT (482476.83023095876 4429

In [527]:
'''The above showed  node 44221 is 9 meters away, here we replace valmont with the Node that belongs to the graph'''
end = (checkNodes[46414].x,checkNodes[46414].y) 

In [1029]:
centr = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0/final_divis_2_0.shp')

In [23]:
'''This is code to find the closest point of any inclusive zone to valmont '''
closetopower = []
fixid = []
for i,p in centr.iterrows():
#     print (type(centr.geometry[i]))
    count = 0
    if type(p.geometry) is shapely.geometry.multipolygon.MultiPolygon:
#         print('fine')
        count = 0
        for k in p.geometry.geoms:
            count+=1
            error_lin, error_zhu = [], []
#             print(type(k.boundary))
            if type(k.boundary) == shapely.geometry.linestring.LineString:
#                 print('ok')
                for s in k.boundary.coords[:]:
                    score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([s,end])] , crs= centr.crs)
                    score_err['error'] = shapely.geometry.LineString([s,end]).length

                    
    #                         score_err['de_centr'] = p.de_centr
                    error_lin.append(score_err)

        #             laos.append(new_.length)
                fixid.append(str(p.neighb)+'_'+str(count))
#                 listid.append(str(p.neighb)+'_'+str(count))
                its_error = pd.concat([l for l in error_lin], ignore_index= True)

                its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
                foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
                foli['neighb'] = p.neighb
                
                if len(foli)>1:
                    print('happened again here')
                    foli = foli.loc[np.where(foli.index == 0)]
                folipnt = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli.geometry[0].coords[:][0])], crs=centr.crs)
                folipnt['neighb'] = p.neighb
                closetopower.append(folipnt)
                
            if type(k.boundary) == shapely.geometry.multilinestring.MultiLineString:
#                 print('noot ok')
            
                for h in k.boundary.geoms[0].coords[:]:
                    score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([h,end])] , crs= simplify_extend.crs)
                    score_err['error'] = shapely.geometry.LineString([h,end]).length
                    
                    error_zhu.append(score_err)

                its_error = pd.concat([l for l in error_zhu], ignore_index= True)
                
                fixid.append(str(p.neighb)+'_'+str(count))
                
                its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
                foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
                
                if len(foli)>1:
                    print('happened')
                    foli = foli.loc[np.where(foli.index == 0)]
                    
                foli['neighb'] = p.neighb
                folipnt = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli.geometry[0].coords[:][0])], crs=centr.crs)
                folipnt['neighb'] = p.neighb
                closetopower.append(folipnt)
                    
    if type(p.geometry) is not shapely.geometry.multipolygon.MultiPolygon:
        error_lin, error_zhu = [], []
        countt = 0
        if type(p.geometry.boundary) == shapely.geometry.linestring.LineString:
            print(p.neighb)
            countt+=1
            for j in p.geometry.boundary.coords[:]:
                score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([j,end])] , crs= centr.crs)
                score_err['error'] = shapely.geometry.LineString([j,end]).length


#                         score_err['de_centr'] = p.de_centr
                error_lin.append(score_err)

            fixid.append(str(p.neighb)+'_'+str(countt))
#                 listid.append(str(p.neighb)+'_'+str(count))
            its_error = pd.concat([l for l in error_lin], ignore_index= True)

            its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
            foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
            foli['neighb'] = p.neighb

            if len(foli)>1:
                print('happened for a single polygon without multiline')
                foli = foli.loc[np.where(foli.index == 0)]
            folipnt = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli.geometry[0].coords[:][0])], crs=centr.crs)
            folipnt['neighb'] = p.neighb
            closetopower.append(folipnt)
            
            
        if type(p.geometry.boundary) == shapely.geometry.multilinestring.MultiLineString:
            print('this was a multilinestring for single polygon', p.neighb)
            countt+=1
            for jo in p.geometry.boundary.geoms[0].coords[:]:
                score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([jo,end])] , crs= simplify_extend.crs)
                score_err['error'] = shapely.geometry.LineString([jo,end]).length

                error_zhu.append(score_err)

            its_error = pd.concat([l for l in error_zhu], ignore_index= True)

            fixid.append(str(p.neighb)+'_'+str(countt))

            its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
            foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)

            if len(foli)>1:
                print('happened for a single polygon with multiline')
                foli = foli.loc[np.where(foli.index == 0)]

            foli['neighb'] = p.neighb
            folipnt = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli.geometry[0].coords[:][0])], crs=centr.crs)
            folipnt['neighb'] = p.neighb
            closetopower.append(folipnt)

happened again here
happened again here
this was a multilinestring for single polygon 5
happened again here
11
happened again here
happened again here
happened again here
happened again here
happened again here
happened
happened again here
happened again here
happened
happened again here
happened again here
happened again here
happened again here
happened again here
happened again here
this was a multilinestring for single polygon 54
this was a multilinestring for single polygon 55


In [24]:
len(closetopower)

625

In [25]:
closetopw = pd.concat([l for l in closetopower])

In [26]:
closetopw['de_centr'] = fixid

In [679]:
closetopw.to_file('PNT_polygon_closetoplant')

In [1040]:
decent,listid,new_fid, decentroid,areade = [],[],[],[],[]
for i,p in centr.iterrows():
#     print (type(centr.geometry[i]))
    count = 0
    countt = 0
    if type(p.geometry) is shapely.geometry.multipolygon.MultiPolygon:
#         print('fine')
        
        for k in list(p.geometry.geoms):
#             print(k)
            count+=1
            core = gpd.GeoDataFrame(geometry = [k], crs = centr.crs)
        
            cocent = gpd.GeoDataFrame(geometry = [k.centroid], crs = centr.crs)
            decentroid.append(cocent)
#             print(k.centroid, p.neighb, count)
            decent.append(core)
            listid.append(str(p.neighb)+'_'+str(count))
            new_fid.append(p.neighb)
            areade.append(k.area/4046)
            
            
    if type(p.geometry) is not shapely.geometry.multipolygon.MultiPolygon:
#         print(p.geometry.centroid,'                YES', p.neighb)
        coree = gpd.GeoDataFrame(geometry = [p.geometry], crs= centr.crs)
        decent.append(coree)
        countt+=1
        listid.append(str(p.neighb)+'_'+str(countt))
        print('lol', p.neighb)
        cocentt = gpd.GeoDataFrame(geometry = [p.geometry.centroid], crs = centr.crs)
        decentroid.append(cocentt)
        new_fid.append(p.neighb)
        areade.append(p.geometry.area/4046)
#         print(centr.geometry[i].centroid)

lol 5
lol 12
lol 13


In [1039]:
len(areade)

625

In [1041]:
dee = pd.concat([i for i in decent])
decentr = pd.concat([i for i in decentroid])

decentr['de_centr'] = listid
decentr['new_fid'] = new_fid
dee['de_centr'] = listid
dee['new_fid'] = new_fid
dee['Acres'] = areade

In [1042]:
decentr.reset_index(drop=True,inplace=True)

In [1047]:
dee.to_file('Multipolygons_areafragmentation')

In [395]:
from shapely.geometry import Point

In [484]:

for g in inxx:
#     print(source.geometry[inxx])
    if type(source.geometry[g].boundary) == shapely.geometry.multilinestring.MultiLineString:
        for gg in source.geometry[g].boundary.geoms[0].coords[:]:
            print(gg)
    
    

(488775.4894000003, 4451224.1055)
(488775.4894000003, 4451251.3672)
(488739.1403999999, 4451251.3672)
(488739.1403999999, 4451242.2798999995)
(488748.2276999997, 4451242.2798999995)
(488748.2276999997, 4451215.018200001)
(488757.31489999965, 4451215.018200001)
(488757.31489999965, 4451224.1055)
(488775.4894000003, 4451224.1055)
(488775.4894000003, 4451187.7565)
(488784.57660000026, 4451187.7565)
(488784.57660000026, 4451196.843800001)
(488793.6638000002, 4451196.843800001)
(488793.6638000002, 4451087.7969)
(488784.57660000026, 4451087.7969)
(488784.57660000026, 4451105.9714)
(488775.4894000003, 4451105.9714)
(488775.4894000003, 4451096.884199999)
(488675.52979999967, 4451096.884199999)
(488675.52979999967, 4451087.7969)
(488666.4425999997, 4451087.7969)
(488666.4425999997, 4451060.5352)
(488639.18090000004, 4451060.5352)
(488639.18090000004, 4451087.7969)
(488575.57019999996, 4451087.7969)
(488575.57019999996, 4451078.7097)
(488484.6979, 4451078.7097)
(488484.6979, 4451105.9714)
(48845

In [488]:
keeppoly,fid = [],[]
for i,y in centr.iterrows():
    source = dee.loc[np.where(dee['new_fid']==y['neighb'])]['geometry']
    if len(source) == 1:
        error_zhu = []
        inex = source.index[0]
        print('one poly')
        if type(dee.loc[inex].geometry.boundary) == shapely.geometry.multilinestring.MultiLineString:
            for d in dee.loc[inex].geometry.boundary.geoms[0].coords[:]:
                score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([d,end])], crs= simplify_extend.crs)
                score_err['error'] = shapely.geometry.LineString([d,end]).length

                error_zhu.append(score_err)

            its_error = pd.concat([l for l in error_zhu], ignore_index= True)

    #         fid.append()

            its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
            foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
            keeppoly.append(gpd.GeoDataFrame(geometry =[Point(foli.geometry[0].coords[:][0])], crs = simplify_extend.crs))
            fid.append(y.neighb)
        if type(dee.loc[inex].geometry.boundary) == shapely.geometry.linestring.LineString:
            for d in dee.loc[inex].geometry.boundary.coords[:]:
                score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([d,end])], crs= simplify_extend.crs)
                score_err['error'] = shapely.geometry.LineString([d,end]).length

                error_zhu.append(score_err)

            its_error = pd.concat([l for l in error_zhu], ignore_index= True)

    #         fid.append()

            its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
            foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
            keeppoly.append(gpd.GeoDataFrame(geometry =[Point(foli.geometry[0].coords[:][0])], crs = simplify_extend.crs))
            fid.append(y.neighb)
    if len(source) >1:
        error_zhu = []
#         print('ok')
        inxx = source.index
        for g in inxx:
            if type(source.geometry[g].boundary) == shapely.geometry.multilinestring.MultiLineString:
                for gg in source.geometry[g].boundary.geoms[0].coords[:]:
                    score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([gg,end])] , crs= simplify_extend.crs)
                    score_err['error'] = shapely.geometry.LineString([gg,end]).length

                    error_zhu.append(score_err)

                its_error = pd.concat([l for l in error_zhu], ignore_index= True)

        #         fid.append()

                its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
                foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
            if type(source.geometry[g].boundary) == shapely.geometry.linestring.LineString:
                for gg in source.geometry[g].boundary.coords[:]:
                    score_err = gpd.GeoDataFrame(geometry = [shapely.geometry.LineString([gg,end])] , crs= simplify_extend.crs)
                    score_err['error'] = shapely.geometry.LineString([gg,end]).length

                    error_zhu.append(score_err)

                its_error = pd.concat([l for l in error_zhu], ignore_index= True)

        #         fid.append()

                its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
                foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
        fid.append(y.neighb)
        keeppoly.append(gpd.GeoDataFrame(geometry =[Point(foli.geometry[0].coords[:][0])], crs = simplify_extend.crs))
#         print(its_err)
        
#         for g in source.contains(shapely.geometry.Point(foli.geometry[0].coords[:][0])):
#         #     print(i.index)
#             if (g==True):
#                 print(inxx[g])
#                 dede = dee.loc[np.where(dee.index==inxx)]
#         keeppoly.append(dede)

one poly
one poly
one poly
one poly


In [490]:
len(keeppoly)

53

In [492]:
keepit = pd.concat(keeppoly)

In [494]:
keepit.reset_index(drop=True,inplace=True)

In [496]:
keepit['neighb'] = fid

In [498]:
keepit.to_file('CLOSESTSSIDEOFPOLY')

In [24]:
perfallo = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0\
/farm_allocation_distributedrandomly/farm_allocation_distributedrandomly.shp')

In [31]:
np.where(decentr['de_centr']=='5_1')

(array([58], dtype=int64),)

In [183]:
error_lin = []
laos=[]
for i in list(np.where(buffclosest.contains(detailnodes)==True))[0]:
#     print(i)
    new_= shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
                                    (detailnodes[i].x,\
                                        detailnodes[i].y)])
    score_err = gpd.GeoDataFrame(geometry = [new_] , crs= simplify_extend.crs)
    score_err['error'] = new_.length
# #                 score_err['APN'] = j.APN
# #                 score_err['farm_siz'] = j.farm_siz
# #                 score_err['new_fid'] = j.new_fid
    score_err['neighb'] = j.new_fid
    score_err['de_centr'] = j.de_centr
    error_lin.append(score_err)
    
    laos.append(new_.length)
its_error = pd.concat([k for k in error_lin], ignore_index= True)

its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 

In [103]:
indx = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].index[0]

In [179]:

foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)

In [202]:
list(np.where(buffclosest.contains(detailnodes)==True))[0]

array([   15,    16,    17, ..., 31196, 31197, 31198], dtype=int64)

In [29]:
closerpolypnt = gpd.read_file('PNT_polygon_closetoplant/PNT_polygon_closetoplant.shp')

In [30]:
closerpolypnt

,neighb,de_centr,geometry
0,1,1_1,POINT (460686.849 4446198.865)
1,1,1_2,POINT (458914.838 4448834.163)
2,1,1_3,POINT (463849.206 4447498.339)
3,1,1_4,POINT (461150.297 4449397.571)
4,1,1_5,POINT (456361.325 4449815.584)
...,...,...,...
620,57,57_7,POINT (479933.611 4450751.569)
621,57,57_8,POINT (482050.936 4448788.727)
622,57,57_9,POINT (476616.770 4451487.635)
623,57,57_10,POINT (473836.076 4440055.895)


In [648]:
decentr.to_file('Declustered_centr')

In [ ]:
centr['geometry'][0].boundary.geoms[0].coords[:]

In [32]:
    '''As above, instead of having the farm itself as a starting point, we use the farm
    and create an increasing buffer until we find the closest node to that farm. Then we use that node as the start'''
    buffname=[]
    buffcontainn = []
    starting = []
    lin_finale = []
    nodlist = []
    # for i,j in points_output.loc[np.where(points_output['boot']==482)].iterrows():
    for i,j in closerpolypnt.iterrows():
        buffcontain=[]
        buffsize=8000 #based on previous run the maximum observed distance between de_centr and nearest node was around 3km
        error_lin =[]
        
        buffclosest = j['geometry'].centroid.buffer(buffsize)
        
        for k in list(np.where(buffclosest.contains(detailnodes)==True))[0]:
        #     print(i)
            new_= shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
                                            (detailnodes[k].x,\
                                                detailnodes[k].y)])
            score_err = gpd.GeoDataFrame(geometry = [new_] , crs= simplify_extend.crs)
            score_err['error'] = new_.length
        # #                 score_err['APN'] = j.APN
        # #                 score_err['farm_siz'] = j.farm_siz
        # #                 score_err['new_fid'] = j.new_fid
            score_err['neighb'] = j.neighb
            score_err['de_centr'] = j.de_centr
            error_lin.append(score_err)

#             laos.append(new_.length)
        its_error = pd.concat([l for l in error_lin], ignore_index= True)

        its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
        foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
        
        if len(foli) == 1: #there are times that nodes have same length around a centroid and minimum returns a list
        
            nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli['geometry'][0].coords[:][1])],\
                                           crs = simplify_extend.crs)

            nod['neighb'] = j.neighb
            nod['de_centr'] = j.de_centr
            nodlist.append(nod)
            lin_finale.append(foli)
            print(foli.error,j.de_centr)
            buffcontainn.append(shapely.geometry.Point(foli['geometry'][0].coords[:][1]))
        
        if len(foli)>1:
            
            folii = foli.loc[np.where(foli.index==0)]
            
            nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(folii['geometry'][0].coords[:][1])],\
                                           crs = simplify_extend.crs)

            nod['neighb'] = j.neighb
            nod['de_centr'] = j.de_centr
            nodlist.append(nod)
            lin_finale.append(folii)
            print(folii.error , j.de_centr)
            buffcontainn.append(shapely.geometry.Point(folii['geometry'][0].coords[:][1]))
            
            
#         while buffcontain == []:

        #     buffclosest = thepoint_inside[indx].buffer(buffsize)
#             buffclosest = j['geometry'].centroid.buffer(buffsize)
            
#             if buffclosest.contains(detailnodes) == True:
#                 print('caught',j.de_centr)
#         for k in range(len(detailnodes)):
            
#             if buffclosest.contains(detailnodes[k])==True:
                
#                 buffcontain.append(detailnodes[k])
#                 print(len(buffcontain))
#                 print(j['de_centr'])
#                 new_=shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
#                                                  (detailnodes[k].x,detailnodes[k].y)])
#                 nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(detailnodes[k].x,detailnodes[k].y)],\
#                                        crs = simplify_extend.crs)
#                 score_err = gpd.GeoDataFrame(geometry = [new_] , crs= simplify_extend.crs)
#                 score_err['error'] = new_.length
# #                 score_err['APN'] = j.APN
# #                 score_err['farm_siz'] = j.farm_siz
# #                 score_err['new_fid'] = j.new_fid
#                 score_err['neighb'] = j.new_fid
#                 score_err['de_centr'] = j.de_centr
#                 error_lin.append(score_err)
#                 nod['neighb'] = j.new_fid
#                 nod['de_centr'] = j.de_centr
#                 nodlist.append(nod)
                
                
#         buffsize+=1
#     buffcontainn.append(nod.geometry[0])

0    46.827379
Name: error, dtype: float64 1_1
0    313.564158
Name: error, dtype: float64 1_2
0    93.467935
Name: error, dtype: float64 1_3
0    2055.18702
Name: error, dtype: float64 1_4
0    522.943698
Name: error, dtype: float64 1_5
0    325.655294
Name: error, dtype: float64 1_6
0    641.855151
Name: error, dtype: float64 1_7
0    2910.520055
Name: error, dtype: float64 1_8
0    2459.230748
Name: error, dtype: float64 1_9
0    56.049317
Name: error, dtype: float64 1_10
0    54.873528
Name: error, dtype: float64 1_11
0    53.996195
Name: error, dtype: float64 1_12
0    46.789036
Name: error, dtype: float64 1_13
0    44.186717
Name: error, dtype: float64 1_14
0    708.399849
Name: error, dtype: float64 1_15
0    54.707235
Name: error, dtype: float64 1_16
0    60.711832
Name: error, dtype: float64 1_17
0    54.60074
Name: error, dtype: float64 1_18
0    490.661769
Name: error, dtype: float64 2_1
0    55.137132
Name: error, dtype: float64 2_2
0    46.616703
Name: error, dtype: float6

KeyboardInterrupt: 

In [33]:
noding = pd.concat([k for k in nodlist] , ignore_index = True)
nodin = gpd.GeoDataFrame(noding, geometry = noding['geometry'], crs = simplify_extend.crs)

In [684]:
nodin.to_file('closestpnt_frompolygonsto_10mrs')

In [33]:
nodin = gpd.read_file('closestpnt_frompolygonsto_10mrs/closestpnt_frompolygonsto_10mrs.shp')

In [34]:
its_error = pd.concat([k for k in lin_finale], ignore_index= True)

its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 

In [686]:
'''This was calculated to identify the closest 10ms point to the closest polygon point towards the power plant'''
its_err.to_file('centerror_10mrs_topolycloserVml')

In [34]:
its_err = gpd.read_file('centerror_10mrs_topolycloserVml/centerror_10mrs_topolycloserVml.shp')

In [35]:
nodin

,neighb,de_centr,geometry
0,1,1_1,POINT (460733.669 4446198.039)
1,1,1_2,POINT (458761.487 4448560.656)
2,1,1_3,POINT (463927.160 4447446.770)
3,1,1_4,POINT (459652.072 4447990.761)
4,1,1_5,POINT (456251.974 4449304.201)
...,...,...,...
620,57,57_7,POINT (480164.418 4451139.242)
621,57,57_8,POINT (482039.634 4448734.387)
622,57,57_9,POINT (476429.841 4451629.920)
623,57,57_10,POINT (473852.931 4440012.693)


In [36]:
# '''This was done to correct the decenter values like from the old ones of 55_55 to 55_1'''
# # points_output.loc[np.where(points_output['de_center']=='55_55'),'de_center'][101] = '55_1'
# for i in list(np.where(points_output['de_center']=='54_54'))[0]: #this is how to update single rows values
#     print(i)
#     points_output.loc[i,'de_center'] = '54_1'
    
# for i in list(np.where(points_output['de_center']=='55_55'))[0]: #this is how to update single rows values
#     print(i)
#     points_output.loc[i,'de_center'] = '55_1'
    
# for i in list(np.where(points_output['de_center']=='5_5'))[0]: #this is how to update single rows values
#     print(i)
#     points_output.loc[i,'de_center'] = '5_1'

28
29
156
213
232
235
304
320
351
515
641
778
825
836
846
921
940
944
975
976
1032
1061
1249
1350
1451
1506
1507
1644
1703
1736
1750
1764
1803
1836
1986
1987
2014
2226
2308
2403
2449
2455
2611
2711
2740
2750
2756
2820
2833
3018
3019
3043
3151
3222
3276
3282
3358
3433
3576
3679
3734
3748
3771
3922
4005
4012
4119
4402
4416
4571
4730
4851
4891
4898
4902
4903
5065
5329
5396
5549
5677
5688
5699
5748
5757
5767
5889
5890
6122
6291
6541
6542
6656
6686
6713
6733
6735
6773
6807
6902
7019
7071
7111
7214
7271
7279
7370
7371
7372
7373
7493
7779
7978
8081
8122
8129
8204
8302
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8500
8516
8750
8759
8836
8837
8993
9411
9412
9413
9414
9590
9744
9827
9857
9955
9987
10011
10017
10077
10102
10164
10327
10328
10329
10350
10414
10691
10888
10889
10890
10891
10892
11304
11436
11520
11532
11575
11638
11641
11707
11712
11723
11768
11778
11929
12244
12360
12361
12362
12363
12364
12365
12366
12404
12440
12837
12838
12839
12840
12920
13185
13232
13412
13593
13643
136

86500
86501
86521
86537
86562
86776
87013
87014
87022
87027
87030
87063
87104
87109
87141
87161
87175
87239
87240
87241
87299
87383
87665
87730
87731
87941
87950
87980
87990
87998
88203
88268
88275
88399
88609
88777
88841
88846
88865
88871
88897
89006
89074
89103
89161
89188
89189
89351
89640
89687
89761
89975
90028
90034
90043
90077
90340
90383
90416
90501
90513
90516
90566
90579
90635
90767
90772
90883
90894
90902
90932
90952
91042
91075
91138
91150
91259
91391
91443
91453
91476
91477
91566
91585
91613
91638
91777
91805
91819
91899
91948
91990
92084
92169
92170
92220
92272
92282
92314
92318
92584
92653
92654
92680
92752
92817
92818
92893
93141
93287
93402
93539
93545
93551
93605
93731
93834
93881
93922
93951
93971
93988
94017
94029
94037
94120
94285
94302
94463
94545
94577
94701
94806
94842
94877
94918
94941
94945
95144
95286
95310
95335
95371
95408
95411
95413
95419
95423
95476
95510
95525
95664
95665
95666
95667
95668
95669
95670
95671
95672
95743
95780
95783
96001
96118
96119
9612

163226
163227
163228
163229
163302
163613
163728
163746
163783
163893
163906
163910
163923
163945
163950
164180
164181
164251
164305
164306
164313
164326
164574
164725
164812
164983
165064
165104
165255
165344
165492
165513
165515
165531
165565
165585
165621
165756
165760
165780
165784
165785
165793
165819
166201
166275
166334
166348
166361
166385
166430
166498
166589
166672
166673
166674
166675
166760
167079
167102
167202
167270
167373
167408
167413
167415
167458
167612
167764
167772
167906
167965
168035
168054
168123
168224
168321
168340
168362
168412
168467
168529
168617
168716
168754
168873
168913
168921
168969
168999
169030
169038
169056
169175
169197
169311
169433
169511
169524
169536
169583
169735
169746
169835
169878
169888
169907
169961
169986
170014
170042
170115
170127
170180
170578
170579
170669
170674
170678
170696
170718
170761
170772
171015
171193
171303
171487
171504
171675
171693
171699
171721
171787
171825
171834
172070
172216
172227
172287
172368
172386
172434
172491

239828
239849
239877
239961
239992
239994
240044
240074
240076
240104
240292
240340
240353
240371
240406
240409
240430
240597
240654
240776
240800
240854
240893
241015
241097
241330
241344
241417
241467
241471
241537
241575
241844
242016
242031
242057
242132
242133
242134
242135
242277
242491
242595
242596
242597
242710
242854
242986
243186
243187
243188
243189
243190
243191
243192
243193
243194
243195
243196
243197
243433
243434
243538
243539
243693
243929
244076
244109
244221
244251
244258
244300
244302
244361
244422
244425
244439
244536
244537
244538
244652
244662
244744
244832
244986
245011
245105
245163
245197
245273
245338
245512
245596
245638
245646
245690
245745
245773
245798
245800
246001
246002
246055
246182
246258
246272
246286
246377
246516
246517
246518
246519
246585
246908
246988
246989
246990
247092
247096
247114
247137
247387
247704
247716
247742
247747
247774
247818
247830
247854
248027
248028
248066
248110
248190
248206
248294
248327
248331
248517
248568
248589
248592

312689
312690
312757
313034
313171
313182
313195
313318
313326
313460
313485
313521
313669
313670
313671
313672
313823
314080
314104
314149
314266
314268
314292
314358
314501
314632
314633
314634
314635
314769
315092
315093
315212
315237
315267
315286
315333
315435
315450
315570
315672
315755
315894
315902
315914
315968
315997
316129
316130
316131
316132
316234
316517
316558
316559
316560
316738
316966
317159
317234
317246
317304
317315
317344
317361
317392
317401
317413
317530
317656
317712
317813
317818
317833
317861
317874
318038
318039
318085
318154
318183
318188
318195
318415
318515
318777
318792
318806
318843
318897
318925
318943
318970
318987
319048
319049
319102
319132
319133
319134
319142
319391
319482
319500
319653
319706
319810
319825
319877
319995
320239
320283
320329
320374
320375
320397
320425
320499
320500
320526
320671
320780
320783
320824
320935
320997
320998
320999
321000
321123
321361
321559
321594
321596
321661
321703
321842
321869
321890
321925
321961
321962
322017

...

129326
129327
129334
129449
129499
129519
129618
129625
129702
129757
129790
129793
129836
129838
129884
129997
130152
130200
130208
130270
130296
130330
130379
130531
130556
130608
130614
130633
130636
130654
130692
130777
130794
130795
130965
131075
131140
131175
131179
131243
131257
131406
131491
131534
131548
131577
131578
131591
131677
131707
131729
131744
131749
131880
132004
132030
132130
132132
132157
132255
132272
132294
132300
132519
132556
132601
132632
132634
132645
132726
132812
132813
132954
132977
132979
132999
133000
133049
133208
133286
133599
133646
133651
133653
133720
133925
134010
134015
134119
134123
134139
134190
134210
134344
134382
134431
134435
134450
134527
134555
134580
134626
134648
134796
134797
134798
134856
135180
135217
135290
135291
135292
135293
135660
135708
135711
135809
135848
135912
135952
135961
135971
136026
136031
136075
136134
136163
136243
136454
136482
136521
136561
136593
136606
136623
136729
136799
136850
136856
136881
136989
136991
137016

210154
210199
210285
210286
210287
210311
210617
210655
210675
210732
210762
210869
210891
210895
210964
211000
211033
211104
211213
211295
211423
211432
211438
211452
211475
211505
211598
211688
211768
211769
212109
212152
212200
212325
212347
212360
212414
212440
212514
212562
212566
212640
212659
212672
212873
212882
212912
213094
213169
213253
213278
213382
213425
213464
213470
213499
213608
213668
213879
213981
214022
214030
214080
214108
214180
214220
214330
214347
214350
214367
214411
214431
214526
214659
214660
214683
214783
214840
214888
215016
215028
215145
215146
215290
215319
215327
215328
215343
215388
215565
215604
215657
215734
215762
215792
216007
216013
216105
216106
216519
216533
216650
216775
216789
216790
216792
216842
216897
216933
216960
217003
217014
217081
217142
217223
217265
217332
217516
217555
217570
217571
217957
217970
218148
218149
218150
218151
218522
218536
218545
218806
218843
218869
218896
218920
219049
219214
219236
219280
219308
219322
219368
219445

286970
286980
287017
287076
287124
287146
287147
287559
287620
287638
287714
287732
288110
288161
288281
288326
288351
288407
288417
288434
288441
288464
288471
288616
288727
288781
288832
288851
288853
288913
289079
289206
289280
289355
289356
289366
289392
289435
289437
289444
289618
289619
289620
289621
289987
290075
290120
290121
290267
290352
290362
290402
290438
290451
290517
290611
290612
290613
290643
290989
291026
291062
291195
291226
291309
291360
291402
291419
291481
291636
291637
291638
291639
291948
291990
291993
292100
292101
292112
292126
292467
292481
292545
292747
292751
292765
292769
292776
292790
292927
293021
293181
293223
293241
293297
293355
293356
293434
293520
293574
293607
293917
293926
294012
294176
294193
294222
294293
294311
294344
294509
294510
294654
294723
294817
294823
294944
294953
294968
294990
295096
295206
295237
295275
295314
295346
295392
295494
295536
295645
295661
295675
295720
295733
295770
295893
295973
295974
296395
296399
296473
296677
296761

367021
367022
367476
367519
367744
367761
367869
367881
367901
367937
367943
367956
368015
368016
368017
368018
368409
368434
368486
368698
368706
368710
368740
368771
368779
368891
368986
369069
369114
369134
369180
369387
369432
369487
369584
369602
369649
369655
369661
369678
369796
369903
369906
369921
369944
369946
369959
370015
370104
370162
370256
370345
370368
370441
370632
370672
370675
370692
370694
370714
370836
370955
370977
371052
371084
371101
371140
371143
371160
371349
371509
371510
371511
371512
371823
371892
371968
371969
371970
371971
372309
372351
372356
372403
372563
372637
372644
372662
372672
372679
372821
372898
373132
373209
373258
373308
373368
373373
373446
373609
373650
373720
373751
373754
373768
373910
374005
374081
374082
374134
374150
374157
374194
374295
374383
374384
374385
374522
374736
374804
374907
374908
374909
374910
375249
375268
375275
375344
375345
375753
375760
375876
376029
376048
376081
376096
376104
376133
376156
376198
376204
376322
376472

443575
443909
443925
443958
444022
444033
444351
444383
444430
444571
444615
444671
444743
444747
444788
444900
444937
444938
445334
445436
445486
445487
445546
445579
445598
445817
445853
445907
446043
446053
446275
446492
446493
446494
446495
446786
446877
446911
447090
447145
447176
447204
447238
447241
447245
447316
447330
447390
447391
447392
447393
447765
447852
447918
447919
448007
448064
448067
448152
448165
448227
448304
448366
448408
448527
448550
448574
448797
448883
448884
448885
448886
449264
449320
449347
449348
449753
449756
449912
449913
449914
449915
450241
450246
450252
450339
450419
450461
450573
450621
450705
450859
450860
451001
451006
451107
451129
451167
451230
451249
451258
451316
451460
451531
451542
451631
451637
451851
451895
452012
452192
452205
452233
452274
452484
452502
452513
452594
452637
452653
452740
452802
452803
452804
452805
453190
453309
453310
453311
453312
453648
453662
453696
453831
453832
453882
453899
453901
453969
454023
454035
454180
454346

In [38]:
points_output.loc[np.where(points_output['de_center']=='55_1')]

,boot,new_fid,farm_siz,APN,de_center,geometry
101,0,55,499.0,131727000019,55_1,POINT (486716.914 4434724.570)
108,0,55,499.0,131536000031,55_1,POINT (485395.155 4432313.236)
111,0,55,499.0,120516000014,55_1,POINT (486163.505 4433885.957)
113,0,55,179.0,120507000010,55_1,POINT (488647.675 4433302.749)
143,0,55,179.0,157935000003,55_1,POINT (487415.931 4432988.929)
...,...,...,...,...,...,...
489513,999,55,999.0,120100000053,55_1,POINT (485554.927 4431867.437)
489641,999,55,179.0,120507002005,55_1,POINT (485644.061 4432791.304)
489830,999,55,179.0,131513000019,55_1,POINT (486349.257 4433682.315)
489887,999,55,179.0,157513000009,55_1,POINT (486523.984 4432005.223)


In [39]:
nodin

,geometry,neighb,de_centr
0,POINT (460733.669 4446198.039),1,1_1
1,POINT (458761.487 4448560.656),1,1_2
2,POINT (463927.160 4447446.770),1,1_3
3,POINT (459652.072 4447990.761),1,1_4
4,POINT (456251.974 4449304.201),1,1_5
...,...,...,...
620,POINT (480164.418 4451139.242),57,57_7
621,POINT (482039.634 4448734.387),57,57_8
622,POINT (476429.841 4451629.920),57,57_9
623,POINT (473852.931 4440012.693),57,57_10


In [107]:
'''This cell is to connect the points of monte carlo to their representative 
10 meter random point '''
perferror = []
for i,j in points_output.iterrows():
    nod = nodin.loc[np.where(nodin['de_centr']==j['de_center'])]
#     print(nod)
    ne_=shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
                                                 (nod['geometry'].x,nod['geometry'].y)])
    ne = gpd.GeoDataFrame(geometry = [ne_] , crs= simplify_extend.crs)
    ne['error'] = ne.length
    ne['farm_siz'] = j['farm_siz']
    ne['new_fid'] = j['new_fid']
    ne['de_centr'] = j['de_center']
    ne['boot'] = j['boot']
    ne['APN'] = j['APN']
    perferror.append(ne)

In [254]:
'''This is set up in order for you to run errors on specific iterrations, which
is probably not needed anymore since you have the cell above.'''

perferror = []
for i,j in points_output.loc[np.where(points_output['boot']==28)].iterrows():
    nod = nodin.loc[np.where(nodin['de_centr']==j['de_center'])]
#     print(nod)
    ne_=shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
                                                 (nod['geometry'].x,nod['geometry'].y)])
    ne = gpd.GeoDataFrame(geometry = [ne_] , crs= simplify_extend.crs)
    ne['error'] = ne.length
    ne['farm_siz'] = j['farm_siz']
    ne['new_fid'] = j['new_fid']
    ne['de_centr'] = j['de_center']
    ne['boot'] = j['boot']
    ne['APN'] = j['APN']
    perferror.append(ne)

In [108]:
perror = pd.concat([k for k in perferror], ignore_index = True)
error_perfa = gpd.GeoDataFrame(perror, geometry = perror['geometry'], crs = simplify_extend.crs)

In [109]:
error_perfa

,geometry,error,farm_siz,new_fid,de_centr,boot,APN
0,"LINESTRING (467841.039 4437882.251, 468860.744...",1296.926983,1000.0,32,32_14,0,131709000001
1,"LINESTRING (473280.375 4456688.908, 472630.760...",2592.569017,1000.0,5,5_1,0,131900000110
2,"LINESTRING (491883.642 4430492.960, 491215.406...",980.060431,999.0,23,23_17,0,120328000001
3,"LINESTRING (481699.458 4455714.672, 483537.090...",5679.111363,999.0,27,27_4,0,120100000053
4,"LINESTRING (479906.479 4432963.843, 481163.821...",1428.361542,999.0,34,34_6,0,120124000006
...,...,...,...,...,...,...,...
489995,"LINESTRING (458161.117 4434511.410, 460063.493...",1917.966240,49.0,51,51_19,999,157936000001
489996,"LINESTRING (495220.173 4454689.849, 492887.347...",2358.684196,49.0,29,29_5,999,131704000042
489997,"LINESTRING (482757.113 4420114.229, 482873.132...",3359.057916,49.0,18,18_2,999,146534000021
489998,"LINESTRING (461487.352 4446498.650, 463927.160...",2617.554810,49.0,1,1_3,999,146534000022


In [837]:
error_perfa.to_file('ERROR_forboot')

In [39]:
error_perfa = gpd.read_file('ERROR_forboot/ERROR_forboot.shp')

In [1201]:
np.sum(error_perfa.loc[np.where(error_perfa['de_centr']=='57_10')].sample(n=44).error)

299326.88189678063

In [1018]:
np.sum(error_perfa.loc[np.where(error_perfa['de_centr']=='46_3')].sample(n=13).error)

47022.9795203255

In [1141]:
np.sum(error_perfa.loc[np.where(error_perfa['de_centr']=='52_16')].sample(n=27).error)

153315.94085396707

In [1163]:
np.sum(error_perfa.loc[np.where(error_perfa['de_centr']=='55_1')].sample(n=16).error)

51842.09086860758

In [697]:

# erroring = []
# for i in list(error_perfa.de_centr):
#     calcerr = its_error.loc[np.where(its_error['de_centr']==i)]
#     calcerr = calcerr.reset_index(drop=True)
#     print(calcerr['error'][0])
#     erroring.append(calcerr['error'])

In [300]:
np.sum(erroring)

578956.650190284

In [306]:
error_perfa.error.sum()

1066317.154330458

In [308]:
frm = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0/add_farms/newfarms_all.shp')
frm = frm.loc[np.where(frm['keep']==1)]

In [325]:
frmcent = frm.geometry.centroid

In [329]:
frmcent = gpd.GeoDataFrame(geometry = frmcent, crs = frm.crs)
frmcent

,geometry
0,POINT (454533.945 4456787.772)
1,POINT (461113.840 4455425.913)
2,POINT (457827.234 4453308.117)
3,POINT (460092.183 4456012.975)
4,POINT (485164.946 4456677.207)
...,...
485,POINT (494864.560 4441716.818)
486,POINT (490085.556 4451619.594)
487,POINT (485360.430 4420464.438)
488,POINT (493239.044 4428990.609)


In [ ]:
frmcent['Id'] = frm['Id']
frmcent['APN'] = frm['APN']
frmcent['APN2'] = frm['APN2']
frmcent['STATE'] = frm['STATE']
frmcent['COUNTY'] = frm['COUNTY']
frmcent['FIPS'] = frm['FIPS']
frmcent['SIT_HSE_NU'] = frm['SIT_HSE_NU']
frmcent['SIT_DIR'] = frm['SIT_DIR']
frmcent['SIT_STR_NA'] = frm['SIT_STR_NA']
frmcent['acres'] = frm['acres']

In [334]:
frmcent['de_centr'] = 0

In [335]:
'''This step is not needed because some centroids of the farms were falling outside the zones'''


# for m,n in frmcent.iterrows():
#     for i,p in dee.iterrows():
# #             print (p['de_centr'])
#         if p.geometry.contains(n.geometry) :
#             frmcent.loc[m,['de_centr']] = p['de_centr']
# #                 print(p['de_centr'])


In [620]:
# frmcent.to_file('newfrms_wcentrID')

In [584]:
frmcent = gpd.read_file('newfrms_wcentrID/newfrms_wcentrID.shp')

In [541]:
nodin = gpd.read_file('onePOLY_nearestroadnode10ms/onePOLY_nearestroadnode10ms.shp')

In [585]:
'''This cell is to connect the farm centroids with the closest 10 meter random point '''
erroring = []
for i in frmcent.de_centr.unique():
    calcerr = nodin.loc[np.where(nodin['neighb']==int(i))]
    calcerr = calcerr.reset_index(drop=True)
    
    for y,j in frmcent.loc[np.where(frmcent['de_centr']==i)].iterrows():
        print(j.de_centr)
        frmerror = gpd.GeoDataFrame(geometry=[shapely.geometry.LineString([j.geometry,calcerr['geometry'][0]])],\
                        crs=frmcent.crs)
        frmerror['error'] = frmerror.length
        frmerror['APN'] = j.APN
        frmerror['acres'] = j.acres
        frmerror['de_centr'] = j.de_centr
        erroring.append(frmerror)


1
1
1
1
6
6
6
6
6
6
6
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
29
29
29
29
29
29
29
29
47
47
47
47
47
47
47
47
47
47
47
47
47
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
28
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
38
38
38
38
38
38
26
26
26
26
26
26
26
17
17
17
17
41
41
41
41
41
41
41
41
41
41
41
41
41
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
54
54
54
54
44
44
44
44
44
44
44
44
44
44
44
44
23
23
23
23
23
23
23
23
48
48
48
48
48
48
48
45
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
21
21
21
21
18
18
18
18

In [586]:
erroringg = pd.concat([i for i in erroring])

In [587]:
erroringg.to_file('farm_error')
erroringg = gpd.read_file('farm_error/farm_error.shp')
erroringg

,error,APN,acres,de_centr,geometry
0,13244.083310,119702000019,18.987700,1,"LINESTRING (454533.945 4456787.772, 463931.304..."
1,10366.103993,119723000040,199.026001,1,"LINESTRING (454129.367 4450828.432, 463931.304..."
2,7801.258192,119736000002,295.658997,1,"LINESTRING (456186.234 4448389.908, 463931.304..."
3,10017.485503,119712000120,154.539993,1,"LINESTRING (456824.090 4454514.851, 463931.304..."
4,572.348916,119900000002,312.462006,6,"LINESTRING (461113.840 4455425.913, 460546.742..."
...,...,...,...,...,...
485,922.340373,157736000012,99.010101,14,"LINESTRING (484990.660 4419447.279, 485820.239..."
486,2098.413369,157735000006,76.074501,14,"LINESTRING (483763.166 4419435.929, 485820.239..."
487,1842.224015,158117000006,81.886497,37,"LINESTRING (459034.045 4423476.309, 460294.142..."
488,2622.452239,158120000016,85.117302,37,"LINESTRING (458657.975 4422770.720, 460294.142..."


In [564]:
distributedsolu = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0/\
farm_allocation_distributedrandomly/farm_allocation_distributedrandomly.shp')

In [573]:
'''This cell is to calculate the distances of the distributed solution within candidate zones'''
erroring = []
for i in distributedsolu.new_fid.unique():
    calcerr = nodin.loc[np.where(nodin['neighb']==int(i))]
    calcerr = calcerr.reset_index(drop=True)
    
    for y,j in distributedsolu.loc[np.where(distributedsolu['new_fid']==i)].iterrows():
        print(j.new_fid)
        disterr = gpd.GeoDataFrame(geometry=[shapely.geometry.LineString([j.geometry,calcerr['geometry'][0]])],\
                        crs=frmcent.crs)
        disterr['error'] = disterr.length
        disterr['APN'] = j.APN
        disterr['acres'] = j.farm_siz
        disterr['new_fid'] = j.new_fid
        erroring.append(disterr)


51
51
56
56
56
56
56
56
26
26
26
26
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
57
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
47
47
47
4

In [574]:
erroring_distributed = pd.concat([i for i in erroring])

In [577]:
erroring_distributed.reset_index(drop=True,inplace=True)
erroring_distributed

,geometry,error,APN,acres,new_fid
0,"LINESTRING (455607.319 4438952.106, 459652.083...",6648.636027,131709000001,1000.0,51
1,"LINESTRING (459982.897 4432996.976, 459652.083...",754.728891,131900000110,1000.0,51
2,"LINESTRING (494536.127 4437687.635, 491280.677...",3262.559181,120100000001,999.0,56
3,"LINESTRING (492590.940 4439700.800, 491280.677...",2585.096302,157931000008,999.0,56
4,"LINESTRING (492336.161 4442433.329, 491280.677...",5072.005116,120100000002,999.0,56
...,...,...,...,...,...
485,"LINESTRING (461668.000 4438568.139, 463284.721...",4480.991019,157702001001,49.0,22
486,"LINESTRING (461632.674 4439135.593, 463284.721...",3971.622068,157926400004,49.0,22
487,"LINESTRING (459551.847 4439214.935, 463284.721...",5139.264695,157926200004,49.0,22
488,"LINESTRING (460837.378 4443054.340, 463284.721...",2466.525919,120326003001,49.0,22


In [578]:
erroring_distributed.to_file('Distributed_error')

In [592]:
'''This cell is to calculate the distances of centroids from candidate zones'''
erroring = []
for i in inside_centr.neighb.unique():
    calcerr = nodin.loc[np.where(nodin['neighb']==int(i))]
    calcerr = calcerr.reset_index(drop=True)
    
    for y,j in inside_centr.loc[np.where(inside_centr['neighb']==i)].iterrows():
#         print(j.neighb)
        disterr = gpd.GeoDataFrame(geometry=[shapely.geometry.LineString([j.geometry,calcerr['geometry'][0]])],\
                        crs=frmcent.crs)
        disterr['error'] = disterr.length
        disterr['new_fid'] = j.neighb
        erroring.append(disterr)

In [593]:
error_centroids = pd.concat([i for i in erroring])
error_centroids.reset_index(drop=True,inplace=True)
error_centroids

,geometry,error,new_fid
0,"LINESTRING (455940.953 4448716.003, 463931.304...",8089.197924,1
1,"LINESTRING (480321.803 4446598.256, 482319.955...",2762.019746,2
2,"LINESTRING (485710.820 4449524.793, 486841.249...",4718.841346,3
3,"LINESTRING (486064.568 4451949.341, 484902.687...",1978.379218,4
4,"LINESTRING (474183.663 4456299.326, 475358.014...",1467.879466,5
5,"LINESTRING (460801.488 4455029.446, 460546.742...",408.322823,6
6,"LINESTRING (467815.784 4449211.283, 466465.406...",3036.921899,7
7,"LINESTRING (460988.908 4445858.265, 465050.902...",4181.023626,8
8,"LINESTRING (453651.057 4440751.068, 455826.540...",2185.232742,9
9,"LINESTRING (468070.162 4419593.540, 471014.239...",3044.222018,10


In [594]:
error_centroids.to_file('error_fromcentroids_candidate')

In [41]:
'''This cell to build in the strata to the farm from the validation'''
strata,buffsize = [],[]
for i,y in erroringg.iterrows():
#     print(y.acres)
    if y.acres<= 79:
        strata.append(79)
        buffsize.append(190)
    if y.acres> 79 and y.acres<= 179:
        strata.append(179)
        buffsize.append(380)
    if y.acres> 179 and y.acres<= 499:
        strata.append(480)
        buffsize.append(720)
    if y.acres> 499 and y.acres<= 999:
        strata.append(999)
        buffsize.append(1090)
    if y.acres>999:
        strata.append(1000)
        buffsize.append(1110)
        
erroringg['strata'] = strata        
erroringg['buff'] = buffsize

In [42]:
erroringg

,error,APN,acres,de_centr,geometry,strata,buff
0,4049.930403,119702000019,18.9877,1_17,"LINESTRING (454533.945 4456787.772, 457382.302...",79,190
1,823.993539,119712000120,154.5400,1_17,"LINESTRING (456824.090 4454514.851, 457382.302...",179,380
2,1679.575840,119900000002,312.4620,6_4,"LINESTRING (461113.840 4455425.913, 460134.017...",480,720
3,1951.666727,119900000127,38.2812,6_4,"LINESTRING (460092.183 4456012.975, 460134.017...",79,190
4,952.402381,119900000124,376.6020,6_4,"LINESTRING (461006.180 4454444.382, 460134.017...",480,720
...,...,...,...,...,...,...,...
485,654.566288,157514000007,68.7339,42_6,"LINESTRING (493662.265 4423461.841, 493019.838...",79,190
486,461.041417,146536000013,78.6634,23_10,"LINESTRING (494053.264 4429199.425, 493643.106...",79,190
487,1432.100137,120521000004,76.8937,47_6,"LINESTRING (489262.278 4451722.961, 488842.719...",79,190
488,1774.017221,120521000024,115.2480,47_6,"LINESTRING (490085.556 4451619.594, 488842.719...",179,380


In [43]:
erroringg.reset_index(drop=True, inplace=True)

In [44]:
import random

In [ ]:
for ii in range(3):
    
    listbuf = gpd.GeoDataFrame()

    while len(listbuf) != 44:
        try:
            ps = list(erroringg.loc[np.where(erroringg['de_centr']=='57_10')].index)
            random.shuffle(ps,random=None)

            listbuf = gpd.GeoDataFrame()
            for i in ps:
                y = erroringg.loc[np.where(erroringg.index==i)]
            #     y = y[y.index[0]]

                sing = points_output.loc[np.where(points_output['de_center']=='57_10')].sample(n=1)
                sing = sing.buffer(int(y.buff))
                sing = gpd.GeoDataFrame(geometry = [sing[sing.index[0]]], crs=points_output.crs)

            #     gg= points_output.loc[np.where(points_output['de_center']=='57_10')].sample(n=10000).reset_index(drop=True)
                gg= points_output.loc[np.where(points_output['de_center']=='57_10')]
                ggg= [gpd.GeoDataFrame(geometry=[il.buffer(int(y.buff))], crs= points_output.crs) for il in gg.geometry]
                ggbuf = pd.concat([il for il in ggg])
                ggbuf.reset_index(drop=True, inplace=True)


                print(y.buff)
                if len(listbuf) >= 1:
                    indexx = np.where(ggbuf.intersects(listbuf.unary_union)==False)
                    print(indexx)
                    listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


                if len(listbuf)==0:
                    listbuf = pd.concat([sing],ignore_index=True)



                print(len(listbuf))
            #     print(y.buff)
            listbuf = listbuf.set_crs(points_output.crs)
            listbuf['acres'] = [i.area/4046 for i in listbuf.geometry]
        except:
            continue
    print(ii)

In [72]:
ps = list(erroringg.loc[np.where(erroringg['de_centr']=='57_10')].index)
random.shuffle(ps,random=None)

listbuf = gpd.GeoDataFrame()
for i,y in erroringg.loc[np.where(erroringg['de_centr']=='57_10')].iterrows():
    sing = points_output.loc[np.where(points_output['de_center']=='57_10')].sample(n=1)
    sing = sing.buffer(int(y.buff))
    sing = gpd.GeoDataFrame(geometry = [sing[sing.index[0]]], crs=points_output.crs)
    
#     gg= points_output.loc[np.where(points_output['de_center']=='57_10')].sample(n=10000).reset_index(drop=True)
    gg= points_output.loc[np.where(points_output['de_center']=='57_10')]
    ggg= [gpd.GeoDataFrame(geometry=[il.buffer(int(y.buff))], crs= points_output.crs) for il in gg.geometry]
    ggbuf = pd.concat([il for il in ggg])
    ggbuf.reset_index(drop=True, inplace=True)
    
    
    print(y.buff)
    if len(listbuf) >= 1:
        indexx = np.where(ggbuf.intersects(listbuf.unary_union)==False)
        print(indexx)
        listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


    if len(listbuf)==0:
        listbuf = pd.concat([sing],ignore_index=True)
    
    

    print(len(listbuf))
#     print(y.buff)
listbuf = listbuf.set_crs(points_output.crs)
listbuf['acres'] = [i.area/4046 for i in listbuf.geometry]

190
1
190
(array([    0,     1,     2, ..., 10966, 10967, 10968], dtype=int64),)
2


C:\Users\zenus\AppData\Local\Temp\ipykernel_12780\961918552.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


190
(array([    1,     2,     3, ..., 10966, 10967, 10968], dtype=int64),)
3


C:\Users\zenus\AppData\Local\Temp\ipykernel_12780\961918552.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


190
(array([    2,     3,     4, ..., 10966, 10967, 10968], dtype=int64),)
4


...

C:\Users\zenus\AppData\Local\Temp\ipykernel_12780\961918552.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


380
(array([ 2105,  2310,  2482,  2534,  3237,  4040,  4259,  4907,  4927,
        5518,  5555,  6772,  6963,  8568,  8621,  8799,  9387,  9441,
       10133, 10419, 10520], dtype=int64),)
43


C:\Users\zenus\AppData\Local\Temp\ipykernel_12780\961918552.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


380
(array([ 2310,  2482,  2534,  3237,  4259,  5518,  5555,  6772,  6963,
        8621,  8799,  9387,  9441, 10133, 10419, 10520], dtype=int64),)
44


C:\Users\zenus\AppData\Local\Temp\ipykernel_12780\961918552.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listbuf = listbuf.append([ggbuf.loc[indexx[0][0]]],ignore_index= True)


In [73]:
'''You have to pop the first item out because the way the code is set it grabs an extra point at the begining'''
'''You actually fixed that on the cell above by reversing the len(listbuf) check'''
# listbuf = listbuf.loc[listbuf.index[1::]]
# listbuf.reset_index(drop=True,inplace=True)

'You actually fixed that on the cell above by reversing the len(listbuf) check'

In [74]:
'''Throw a small buffer around the points of the selected buffers so that it intersects to the points from simulation
, so that you can tie it back to the orginial points and to the original error from simulation'''
cn = [i.centroid.buffer(1) for i in listbuf.geometry]
cnn = [gpd.GeoDataFrame(geometry = [i], crs = points_output.crs) for i in cn]
cnn= pd.concat(cnn)
cnn.reset_index(drop = True, inplace=True)
# cnn.to_file('varryingbuffer_centroids')

In [75]:
'''This intersects the output of the fitting the buffeers with the points of the simulation'''
varryingoutponts = points_output.loc[np.where(points_output.intersects(cnn.unary_union)==True)]
varryingoutponts.reset_index(drop=True,inplace=True)

In [76]:
'''This intersects the selected points of the simulation back to their respective error'''
varrying_error = error_perfa.loc[np.where(error_perfa.intersects(varryingoutponts.unary_union))]
varrying_error.reset_index(drop=True,inplace=True)

In [77]:
listbuf.to_file('varrying_buffer_FINAL_ex11')
varrying_error.to_file('varrying_error_FINAL_ex11')
varryingoutponts.to_file('varrying_points_FINAL_ex11')

In [845]:
'''This cell is to connect the centroid error to closest 10 meter random point '''
errori = []
for i,j in decentr.iterrows():
    calcerr = nodin.loc[np.where(nodin['de_centr']==j.de_centr)]
    calcerr = calcerr.reset_index(drop=True)

    print(j.de_centr)
    frmerror = gpd.GeoDataFrame(geometry=[shapely.geometry.LineString([j.geometry,calcerr['geometry'][0]])],\
                    crs=frmcent.crs)
    
    frmerror['error'] = frmerror.length
    frmerror['new_fid'] = j.new_fid
    
    frmerror['de_centr'] = j.de_centr
    errori.append(frmerror)


1_1
1_2
1_3
1_4
1_5
1_6
1_7
1_8
1_9
1_10
1_11
1_12
1_13
1_14
1_15
1_16
1_17
1_18
2_1
2_2
2_3
2_4
2_5
2_6
3_1
3_2
3_3
3_4
3_5
3_6
3_7
3_8
3_9
3_10
3_11
3_12
3_13
3_14
3_15
3_16
3_17
3_18
3_19
3_20
3_21
3_22
3_23
3_24
3_25
3_26
3_27
3_28
4_1
4_2
4_3
4_4
4_5
4_6
5_1
6_1
6_2
6_3
6_4
7_1
7_2
7_3
8_1
8_2
8_3
8_4
8_5
8_6
8_7
9_1
9_2
9_3
10_1
10_2
10_3
10_4
10_5
10_6
10_7
10_8
10_9
10_10
10_11
10_12
10_13
11_1
12_1
12_2
14_1
14_2
14_3
14_4
14_5
14_6
15_1
15_2
15_3
15_4
15_5
15_6
15_7
15_8
15_9
15_10
15_11
15_12
15_13
15_14
15_15
15_16
15_17
15_18
15_19
15_20
16_1
16_2
16_3
16_4
16_5
16_6
16_7
16_8
17_1
17_2
17_3
17_4
17_5
18_1
18_2
18_3
18_4
18_5
18_6
18_7
18_8
19_1
19_2
19_3
19_4
19_5
19_6
19_7
19_8
19_9
19_10
19_11
19_12
19_13
19_14
19_15
19_16
19_17
19_18
19_19
19_20
20_1
20_2
20_3
20_4
20_5
20_6
20_7
20_8
20_9
20_10
20_11
20_12
20_13
20_14
20_15
20_16
20_17
20_18
20_19
20_20
20_21
21_1
21_2
21_3
21_4
21_5
21_6
21_7
21_8
21_9
21_10
21_11
21_12
21_13
21_14
21_15
21_16
21_17
21_18
22_1
22_2
2

In [847]:
errorin = pd.concat([i for i in errori])

In [849]:
errorin.to_file('errorfrom_centroid')

In [73]:
probablepnts= gpd.GeoDataFrame(geometry = [shapely.geometry.Point(i) for i in centr['geometry'][0].geoms[0].boundary.coords[:]],\
                               crs = simplify_extend.crs)

In [80]:
keepit = gpd.read_file('CLOSESTSSIDEOFPOLY/CLOSESTSSIDEOFPOLY.shp')

In [618]:
frmcent['neighb'] = frmcent['de_centr']
frmcent

,Id,APN,APN2,STATE,COUNTY,FIPS,SIT_HSE_NU,SIT_DIR,SIT_STR_NA,acres,de_centr,geometry,neighb
0,0,119702000019,168817089,CO,Boulder,08013,8340,NaN,PEAK TO PEAK,18.987700,1,POINT (454533.945 4456787.772),1
1,0,119900000002,168817136,CO,Boulder,08013,3921,NaN,COUNTY ROAD 82,312.462006,6,POINT (461113.840 4455425.913),6
2,0,119900000004,168817647,CO,Boulder,08013,NaN,NaN,COUNTY RD 82E,72.701698,6,POINT (457827.234 4453308.117),6
3,0,119900000127,168817902,CO,Boulder,08013,NaN,NaN,COUNTY RD 82E,38.281200,6,POINT (460092.183 4456012.975),6
4,0,120301000003,168818217,CO,Boulder,08013,8190,N,COUNTY LINE,10.042300,27,POINT (485164.946 4456677.207),27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,0,131524000020,168942713,CO,Boulder,08013,NaN,E,COUNTY LINE,75.119400,56,POINT (494864.560 4441716.818),56
486,0,120521000024,168942777,CO,Boulder,08013,9950,NaN,VERMILLION,115.248001,47,POINT (490085.556 4451619.594),47
487,0,157725000012,168943933,CO,Boulder,08013,2701,NaN,MCCASLIN,92.405098,18,POINT (485360.430 4420464.438),18
488,0,146535100001,168943991,CO,Boulder,08013,NaN,N,119TH,155.097000,23,POINT (493239.044 4428990.609),23


In [641]:
distributedsolu

,boot,new_fid,farm_siz,APN,geometry
0,8,51,1000.0,131709000001,POINT (455607.319 4438952.106)
1,13,51,1000.0,131900000110,POINT (459982.897 4432996.976)
2,3,56,999.0,120100000001,POINT (494536.127 4437687.635)
3,4,56,999.0,157931000008,POINT (492590.940 4439700.800)
4,5,56,999.0,120100000002,POINT (492336.161 4442433.329)
...,...,...,...,...,...
485,36,22,49.0,157702001001,POINT (461668.000 4438568.139)
486,36,22,49.0,157926400004,POINT (461632.674 4439135.593)
487,36,22,49.0,157926200004,POINT (459551.847 4439214.935)
488,36,22,49.0,120326003001,POINT (460837.378 4443054.340)


In [672]:
distra = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0/inside_cent.shp')

In [642]:
perfallo = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/PADUS_2_0\
/farm_allocation_distributedrandomly/farm_allocation_distributedrandomly.shp')

In [644]:
perfallo['neighb'] = perfallo['new_fid']

In [675]:
    '''As above, instead of having the farm itself as a starting point, we use the farm
    and create an increasing buffer until we find the closest node to that farm. Then we use that node as the start'''
    buffname=[]
    buffcontainn = []
    starting = []
    lin_finale = []
    nodlist = []
    # for i,j in points_output.loc[np.where(points_output['boot']==482)].iterrows():
    for i,j in distra.iterrows():
        buffcontain=[]
        buffsize=8000 #based on previous run the maximum observed distance between de_centr and nearest node was around 3km
        error_lin =[]
        
        buffclosest = j['geometry'].centroid.buffer(buffsize)
        
        for k in list(np.where(buffclosest.contains(detailnodes)==True))[0]:
        #     print(i)
            new_= shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
                                            (detailnodes[k].x,\
                                                detailnodes[k].y)])
            score_err = gpd.GeoDataFrame(geometry = [new_] , crs= simplify_extend.crs)
            score_err['error'] = new_.length
        # #                 score_err['APN'] = j.APN
        # #                 score_err['farm_siz'] = j.farm_siz
        # #                 score_err['new_fid'] = j.new_fid
            score_err['neighb'] = j.neighb
#             score_err['de_centr'] = j.de_centr
            error_lin.append(score_err)

#             laos.append(new_.length)
        its_error = pd.concat([l for l in error_lin], ignore_index= True)

        its_err =gpd.GeoDataFrame(its_error,geometry=its_error['geometry'],crs=simplify_extend.crs) 
        foli = its_err.loc[np.where(its_err['error']==np.min(its_err['error']))].reset_index(drop=True)
        
        if len(foli) == 1: #there are times that nodes have same length around a centroid and minimum returns a list
        
            nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(foli['geometry'][0].coords[:][1])],\
                                           crs = simplify_extend.crs)

            nod['neighb'] = j.neighb
#             nod['de_centr'] = j.de_centr
            nodlist.append(nod)
            lin_finale.append(foli)
            print(foli.error,j.neighb)
            buffcontainn.append(shapely.geometry.Point(foli['geometry'][0].coords[:][1]))
        
        if len(foli)>1:
            
            folii = foli.loc[np.where(foli.index==0)]
            
            nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(folii['geometry'][0].coords[:][1])],\
                                           crs = simplify_extend.crs)

            nod['neighb'] = j.neighb
#             nod['de_centr'] = j.de_centr
            nodlist.append(nod)
            lin_finale.append(folii)
            print(folii.error , j.neighb)
            buffcontainn.append(shapely.geometry.Point(folii['geometry'][0].coords[:][1]))
            
            
#         while buffcontain == []:

        #     buffclosest = thepoint_inside[indx].buffer(buffsize)
#             buffclosest = j['geometry'].centroid.buffer(buffsize)
            
#             if buffclosest.contains(detailnodes) == True:
#                 print('caught',j.de_centr)
#         for k in range(len(detailnodes)):
            
#             if buffclosest.contains(detailnodes[k])==True:
                
#                 buffcontain.append(detailnodes[k])
#                 print(len(buffcontain))
#                 print(j['de_centr'])
#                 new_=shapely.geometry.LineString([(j['geometry'].centroid.x,j['geometry'].centroid.y),\
#                                                  (detailnodes[k].x,detailnodes[k].y)])
#                 nod = gpd.GeoDataFrame(geometry = [shapely.geometry.Point(detailnodes[k].x,detailnodes[k].y)],\
#                                        crs = simplify_extend.crs)
#                 score_err = gpd.GeoDataFrame(geometry = [new_] , crs= simplify_extend.crs)
#                 score_err['error'] = new_.length
# #                 score_err['APN'] = j.APN
# #                 score_err['farm_siz'] = j.farm_siz
# #                 score_err['new_fid'] = j.new_fid
#                 score_err['neighb'] = j.new_fid
#                 score_err['de_centr'] = j.de_centr
#                 error_lin.append(score_err)
#                 nod['neighb'] = j.new_fid
#                 nod['de_centr'] = j.de_centr
#                 nodlist.append(nod)
                
                
#         buffsize+=1
#     buffcontainn.append(nod.geometry[0])

0    531.809065
Name: error, dtype: float64 1
0    530.377518
Name: error, dtype: float64 2
0    809.754822
Name: error, dtype: float64 3
0    1202.644299
Name: error, dtype: float64 4
0    964.228325
Name: error, dtype: float64 5
0    323.497943
Name: error, dtype: float64 6
0    710.628514
Name: error, dtype: float64 7
0    262.941133
Name: error, dtype: float64 8
0    1804.029272
Name: error, dtype: float64 9
0    623.780489
Name: error, dtype: float64 10
0    862.919525
Name: error, dtype: float64 14
0    808.759872
Name: error, dtype: float64 15
0    1262.377341
Name: error, dtype: float64 16
0    2391.712588
Name: error, dtype: float64 17
0    1430.592078
Name: error, dtype: float64 18
0    525.998026
Name: error, dtype: float64 19
0    740.401539
Name: error, dtype: float64 20
0    716.075691
Name: error, dtype: float64 21
0    770.585908
Name: error, dtype: float64 22
0    413.808571
Name: error, dtype: float64 23
0    301.459334
Name: error, dtype: float64 24
0    509.186971
N

In [676]:
noding = pd.concat([k for k in nodlist] , ignore_index = True)
nodin = gpd.GeoDataFrame(noding, geometry = noding['geometry'], crs = simplify_extend.crs)

In [677]:
lin_fin = pd.concat(lin_finale)

In [678]:
lin_fin = gpd.GeoDataFrame(lin_fin, crs=simplify_extend.crs)
lin_fin.reset_index(drop=True,inplace=True)

In [662]:
# lin_fin['APN'] = perfallo['APN']
# nodin['APN'] = perfallo['APN']

In [679]:
nodin.to_file('candidatecentroids_nearestroadnode10ms')
lin_fin.to_file('linestring_fromcandidatecentroid_10ms')

In [680]:
how_lin,adj_lin =[] , []
for l,k in nodin.iterrows():
    '''Three routines initiated for different graphs, newGraph, newGraph_adj, newGraph_pix
    this is being done just in case one of these routes will take a different path, thats why we need three.

        aloo: the output geodataframe which contains the route and distance metrics for planar
    aloo_ad: the output geodataframe which contains the route and distance metrics for adjusted
    aloo_ds: the output geodataframe which contains the route and distance metrics for pixel to pixel
    sequence: stores the sequence segments from the start point to end point
    how_lin: combines the three geodataframes 
    '''
    try:
        strt = (k.geometry.x,k.geometry.y)
        showme= nx.dijkstra_path(detailgraph,strt,end)
        showmee=[shapely.geometry.LineString([([showme[i][0],showme[i][1]]), \
                                     (showme[i+1][0],showme[i+1][1])]) for i in range(len(showme)-1)]

        letsee = shapely.geometry.MultiLineString([i for i in showmee])
        p2pD = letsee.length


        alo=[shapely.geometry.Point([i[0],i[1]]) for i in showme]
        aloo = [shapely.geometry.LineString(alo)] 

        aloo = gpd.GeoDataFrame(geometry = aloo,crs=crs)
        aloo['vector'] = p2pD
        aloo['neighb'] = k.neighb
#         aloo['de_centr'] = k.de_centr
    #         aloo['']
        sequence = gpd.GeoDataFrame(geometry = showmee,crs = its_adjusted.crs)
        how_lin.append(aloo)
        print(l)
        
        showme_ad= nx.dijkstra_path(newGraph_adj,strt,end)
        showmee_ad=[shapely.geometry.LineString([([showme_ad[i][0],showme_ad[i][1]]), \
                                     (showme_ad[i+1][0],showme_ad[i+1][1])]) for i in range(len(showme_ad)-1)]

        letsee = shapely.geometry.MultiLineString([i for i in showmee_ad])
        p2pD = letsee.length
        adj_len = nx.dijkstra_path_length(newGraph_adj,strt,end)

        alo_ad=[shapely.geometry.Point([i[0],i[1]]) for i in showme_ad]
        aloo_ad = [shapely.geometry.LineString(alo_ad)] 

        aloo_ad = gpd.GeoDataFrame(geometry = aloo_ad,crs=crs)
        aloo_ad['vector'] = p2pD
        aloo_ad['surface_adjusted'] = adj_len
        aloo_ad['diff_distances'] = adj_len - p2pD
        adj_lin.append(aloo_ad)
    except: 
        print('error')
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [681]:
'''In case the route changes between planar or adjusted or pixel to pixel, this geodataframe will capture that.
Since for this route it doesnt change, it is not saved as an extra output at this point.''' 
how_ = pd.concat([k for k in how_lin], ignore_index = True)
how_

,geometry,vector,neighb
0,"LINESTRING (456381.421 4449014.012, 456387.774...",51458.827374,1
1,"LINESTRING (480321.555 4447128.633, 480331.557...",24882.264091,2
2,"LINESTRING (485710.279 4450334.547, 485700.221...",24147.626062,3
3,"LINESTRING (484861.982 4451961.179, 484861.907...",24912.085775,4
4,"LINESTRING (474269.653 4455338.939, 474279.636...",35443.422472,5
5,"LINESTRING (460499.112 4455144.425, 460494.864...",66082.341153,6
6,"LINESTRING (467190.092 4448874.381, 467190.085...",55220.646288,7
7,"LINESTRING (460938.771 4445600.148, 460928.857...",44957.087220,8
8,"LINESTRING (455356.947 4441337.978, 455356.966...",43822.535266,9
9,"LINESTRING (468084.019 4418969.914, 468094.024...",33527.684506,10


In [682]:
adj_linn = pd.concat(adj_lin)

In [683]:
adj_linn.reset_index(drop=True,inplace=True)

In [684]:
adj_linn['neighb'] = how_.neighb
adj_linn

,geometry,vector,surface_adjusted,diff_distances,neighb
0,"LINESTRING (456381.421 4449014.012, 456387.774...",51458.827374,51597.127062,138.299687,1
1,"LINESTRING (480321.555 4447128.633, 480331.557...",24882.264091,24887.896770,5.632679,2
2,"LINESTRING (485710.279 4450334.547, 485700.221...",24147.626062,24153.773913,6.147851,3
3,"LINESTRING (484861.982 4451961.179, 484861.907...",24912.085775,24918.289126,6.203351,4
4,"LINESTRING (474269.653 4455338.939, 474279.636...",35443.422472,35465.267185,21.844713,5
5,"LINESTRING (460499.112 4455144.425, 460494.864...",66082.341153,66251.883044,169.541891,6
6,"LINESTRING (467190.092 4448874.381, 467190.085...",55220.646288,55304.310564,83.664277,7
7,"LINESTRING (460938.771 4445600.148, 460928.857...",44957.087220,45084.719203,127.631983,8
8,"LINESTRING (455356.947 4441337.978, 455356.966...",43822.535266,43948.555121,126.019854,9
9,"LINESTRING (468084.019 4418969.914, 468094.024...",33527.684506,33649.184041,121.499535,10


In [669]:
# adj_linn['APN'] = perfallo['APN']

In [685]:
adj_linn.to_file('ROUTESfromCANDIDATECENTR') #you need to rerun the centroids for the routes

C:\Users\zenus\AppData\Local\Temp\ipykernel_21472\1192445906.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  adj_linn.to_file('ROUTESfromCANDIDATECENTR') #you need to rerun the centroids for the routes


In [102]:
how_.loc[np.where(how_['vector']==how_['vector'].min())].to_file('exampleofPOLYGONclosesttovalmon')

In [249]:
how_.to_file('test_centr_inclusive_detailed') #final output of Dijkstra routine

In [10]:
%pwd

'C:\\Users\\zenus\\Downloads\\Resilio_box_desktop\\Proposal\\figshare_Dijks'

In [11]:
adjfreq = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
adjusted_frequentlines_insidezones/adjusted_frequentlines_insidezones.shp')

adjfarms = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
adjusted_farmlines_insidezones/adjusted_farmlines_insidezones.shp')

adjzones = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
adjusted_candidatelines_insidezones/adjusted_candidatelines_insidezones.shp')

In [12]:
routesfarms = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
ROUTESfromFARMCENTR\ROUTESfromFARMCENTR.shp')

routeszones = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
ROUTESfromCANDIDATECENTR\ROUTESfromCANDIDATECENTR.shp')

routesfrequency = gpd.read_file('C:/Users/zenus/Downloads/Resilio_box_desktop/Proposal/figshare_Dijks/\
ROUTESfromFREQUENTCENTR\ROUTESfromFREQUENTCENTR.shp')

In [19]:
adjfreq

,planar,surf_adj,std,mean,APN,n_pnts,neighb,geometry
0,1930.419350,1953.884001,3158.603254,2797.416066,131709000001,193,51,"LINESTRING (455607.319 4438952.106, 455607.347..."
1,245.080856,265.085684,261.463678,2581.178306,131900000110,25,51,"LINESTRING (459982.897 4432996.976, 459982.897..."
2,663.856909,663.984842,0.362365,1511.648947,120100000001,22,35,"LINESTRING (494536.127 4437687.635, 494541.601..."
3,1021.986300,1022.125340,6.689954,1550.267891,157931000008,102,35,"LINESTRING (492590.940 4439700.800, 492590.930..."
4,1070.349016,1071.008310,21.266620,1515.332937,120100000002,107,35,"LINESTRING (492336.161 4442433.329, 492336.042..."
...,...,...,...,...,...,...,...,...
485,1636.451081,1656.117322,764.062299,2617.876021,157702001001,164,22,"LINESTRING (461668.000 4438568.139, 461668.003..."
486,1088.894846,1101.723132,397.869221,2633.397716,157926400004,109,22,"LINESTRING (461632.674 4439135.593, 461632.676..."
487,163.249941,163.299519,0.168778,2730.083758,157926200004,6,22,"LINESTRING (459551.847 4439214.935, 459600.503..."
488,3952.132907,3972.540435,316.938974,2652.453138,120326003001,132,22,"LINESTRING (460837.378 4443054.340, 460601.719..."


In [896]:
adjfreq.loc[np.where(adjfreq['neighb']==35)].surf_adj.sum()

4410.308745184813

In [900]:
len(routesfrequency.loc[np.where(routesfrequency['neighb']==39)])

29

In [644]:
pickzone =52

In [645]:
'''Total distance from centroid'''
((routeszones.loc[np.where(routeszones['neighb']==pickzone)].surface_ad.sum()+\
 adjzones.loc[np.where(adjzones['neighb']==pickzone)].surf_adj.sum())*\
len(routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)]))

1508166.1847441786

In [646]:
'''Average dist from centroid'''
((routeszones.loc[np.where(routeszones['neighb']==pickzone)].surface_ad.sum()+\
 adjzones.loc[np.where(adjzones['neighb']==pickzone)].surf_adj.sum())*\
len(routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)]))/\
len(adjfreq.loc[np.where(adjfreq['neighb']==pickzone)])

15875.433523622933

In [647]:
'''Total distance from allocated'''
routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)].surface_ad.sum()+\
adjfreq.loc[np.where(adjfreq['neighb']==pickzone)].surf_adj.sum()

1790640.3587557652

In [648]:
'''Average distance from allocated'''
(routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)].surface_ad.sum()+\
adjfreq.loc[np.where(adjfreq['neighb']==pickzone)].surf_adj.sum())/\
len(adjfreq.loc[np.where(adjfreq['neighb']==pickzone)])

18848.845881639634

In [649]:
len(adjfreq.loc[np.where(adjfreq['neighb']==pickzone)])

95

In [650]:
'''Total distance from validation'''
routesfarms.loc[np.where(routesfarms['neighb']==str(pickzone))].surface_ad.sum()+\
adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))].surf_adj.sum()

625708.9105516835

In [651]:
'''Average distance from validation'''
(routesfarms.loc[np.where(routesfarms['neighb']==str(pickzone))].surface_ad.sum()+\
adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))].surf_adj.sum())/\
len(adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))])

20184.158404893016

In [652]:
len(adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))])

31

In [643]:
adjfreq.loc[np.where(adjfreq['neighb']==pickzone)].surf_adj.sum()/len(adjfreq.loc[np.where(adjfreq['neighb']==pickzone)])

1109.4848426038268

In [504]:
adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))].surf_adj.sum()/\
len(adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))])

1088.367259671409

In [505]:
(adjzones.loc[np.where(adjzones['neighb']==pickzone)].surf_adj.sum()*\
len(routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)]))/\
len(adjfreq.loc[np.where(adjfreq['neighb']==pickzone)])

334.30863366401934

In [506]:
adjfreq.loc[np.where(adjfreq['neighb']==pickzone)].surf_adj.sum()

34679.46510768802

In [507]:
adjfarms.loc[np.where(adjfarms['neighb']==str(pickzone))].surf_adj.sum()

14148.774375728317

In [508]:
adjzones.loc[np.where(adjzones['neighb']==pickzone)].surf_adj.sum()*\
len(routesfrequency.loc[np.where(routesfrequency['neighb']==pickzone)])

10697.876277248619